In [ ]:
#### A FAIRE pour présenter à Hervé

# - comment paramétrer :
#     - un titre particulier à découper
#     - l'ajout de corpus = ajout de descriptions ?
                                                                                
#   Le principe du chunking est présent dans la plupart des bouquins sur le NLP
#   exemple : "Oreilly chunking with Python"
# - diagnostic : cause des vins pas du tout découpés ?
# - est-ce que les stopwords servent vraiment ?
# - les résultats obtenus 
# - bémol sur les résultats car ce modèle est évalué sur des titres qui ont eu la chance d'avoir un découpage
# parmi les propriétés des vins
# On voit que la plupart possède en fait une seule entité dans leurs titres


# BUT : si améliorer le % d'entités retrouvées dans les titres 
# - Test 1 pas satisfaisant: découper encore plus avec les ADJP pour obtenir un meilleur
# résultat que 76%
    
# - Test 2 en cours : corpus ajouté de 20000 vins du Figaro
# Pourquoi j'ai que 75.8 % ?
# Quelle est l'explication ?
    
# - Si impossible d'améliorer ces 76 %, on peut tester des expressions régulières en NP, ADJP, ADVP et PP
# mais ces expressions régulières seront propres au vin
# Probablement ces expressions régulières seront différentes pour les smartphones

# - Autres solutions :
#     modèle CRF
#     ou Deep learning : RNN cf mooc "named entity recognition"
# Par exemple: le modèle CRF ou DL pourrait prédire 
#     pour une description donnée, les entités présentes
# Et toutes les entités trouvées = vocabulaires d'entités qui servent à découper les titres plus finement que 
# la méthode "chunking" NLP 


# Autre perspective: matcher avec la CV des images de produits
# le vin = extraire les étiquettes
# appliquer un modèle de type DeepFace aux étiquettes

# Ou appliquer juste une méthode de Computer Vision 
# cf méthode de Serge mais l'efficacité dépendrait de la qualité
# des images qui peut différer entre les distributeurs

#### A FAIRE après pour évaluer le modèle sur la vérité
- pour les vins
- les smartphones

pour les smartphones :
pipeline pas prêt car nettoyage à adapter
les stopwords ne sont pas les mêmes que pour le vin
Enfin la fin du pipeline est différente car les 1-grams ont un role différent par rapport au vin

Minhash lsh :
- échantillon vérité de titres tiré parmi 
- les candidats en sortie de Minhash lsh : obtenir un découpage vérité 
- Evaluer le modèle

#### step_0_import_data

In [3]:
# A faire au début, pour utiliser "%autoreload" : 
%load_ext autoreload
%autoreload

import pandas as pd
pd.set_option('display.max_colwidth',-1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ATTENTION : le fichier "wac" des descriptions n'est pas propre pour ce pipeline:

In [53]:
# ([A-Z]|[a-z])  C'
# Une modification de "wine_wac_descriptions.csv"  a été faite pour définir des fins de phrases
# - remplacement : ([A-Z]|[a-z])  C' par ([A-Z]|[a-z]). C' équivalent à remplacer : [a-z]  [A-Z] par [a-z]. [A-Z]
# - remplacement de "Description" par "Description." équivalent à remplacer : [a-z]  [A-Z] par [a-z]. [A-Z]
# - remplacement de "Le domaine" par "Le domaine." équivalent à remplacer : [a-z]  [A-Z] par [a-z]. [A-Z]
# NB) IL RESTE à continuer de remplacer : [A-Z]  [A-Z] par : [A-Z]. [A-Z], en général au début de la description
# mais il y a un risque de faire ce remplacement
# juste avant un nom propre ce qui ne serait pas correct !
# On remarque aussi qu'il y a des phrases au milieu des descriptions ou il manque un "."

# => On essaye de remplacer : ([a-z]|[A-Z])  [A-Z] par [a-z]. [A-Z]

In [4]:
from functions import fun_create_df_cleansing_titles_find_ngrams as fc

# Renseigner le chemin des données source:
data_folder = "/home/hapax94/Documents/vincent/jupyter/carrefour_bio/fichiers_générés_par_Hervé"

orgc_crf_product = fc.create_df("orgc_crf_product",data_folder + "/crf/","utf8")
orgc_crf_product['distributeur'] = 'crf'

orgc_gwz_product = fc.create_df("orgc_gwz_product",data_folder + "/gwz/","utf8")
orgc_gwz_product['distributeur'] = 'gwz'

orgc_ntr_product = fc.create_df("orgc_ntr_product",data_folder + "/ntr/","utf8")
orgc_ntr_product['distributeur'] = 'ntr'

orgc_wbe_product = fc.create_df("orgc_wbe_product",data_folder + "/wbe/","utf8")
orgc_wbe_product['distributeur'] = 'wbe'

name_file: orgc_crf_product
which_file product
name_file: orgc_gwz_product
which_file product
name_file: orgc_ntr_product
which_file product
name_file: orgc_wbe_product
which_file product


In [5]:
# On rassemble tous les produits bio dans un dataframe
import pandas as pd
frames = [orgc_crf_product, orgc_gwz_product, orgc_ntr_product, orgc_wbe_product]
orgc_descriptions = pd.concat(frames)

del frames
del orgc_crf_product, orgc_gwz_product, orgc_ntr_product, orgc_wbe_product 

In [6]:
orgc_descriptions.head()

,xid,xtitle,xbrand,xbreadcrumb,xsubtitle,xdescription,xslug,ximg,xmeta_title,xmeta_description,xmeta_keywords,xapi,xweb,xcrawled_at,xurl,xru,distributeur
0,fromage-bio-frais-carrefour-bio-3245413442420,Fromage bio frais CARREFOUR BIO,Carrefour Bio,"Accueil > Crémerie > Yaourts, desserts et spécialités végétales > Yaourts Nature et Fromages blancs",,"Acheter votre Fromage bio frais CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",fromage-bio-frais-carrefour-bio-3245413442420,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/P.L.S./3245413442420_PHOTOSITE_20160922_080810_0.jpg?placeholder=1,Fromage bio frais CARREFOUR BIO pas cher au Meilleur Prix,"Acheter votre Fromage bio frais CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:53:16,https://www.carrefour.fr/p/fromage-bio-frais-carrefour-bio-3245413442420,19-01-17\n,crf
1,yaourts-bio-nature-carrefour-bio-3245413442338,Yaourts bio nature CARREFOUR BIO,Carrefour Bio,"Accueil > Crémerie > Yaourts, desserts et spécialités végétales > Yaourts Nature et Fromages blancs",,"Acheter votre Yaourts bio nature CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",yaourts-bio-nature-carrefour-bio-3245413442338,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/P.L.S./3245413442338_PHOTOSITE_20161005_084045_0.jpg?placeholder=1,Yaourts bio nature CARREFOUR BIO pas cher au Meilleur Prix,"Acheter votre Yaourts bio nature CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:53:22,https://www.carrefour.fr/p/yaourts-bio-nature-carrefour-bio-3245413442338,19-01-17\n,crf
2,cafe-bio-equilibre-carrefour-bio-3560070712786,Café bio équilibré CARREFOUR BIO,Carrefour Bio,Accueil > Epicerie Sucrée > Boissons du matin et du soir > Cafés moulus et en grains,,"Acheter votre Café bio équilibré CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",cafe-bio-equilibre-carrefour-bio-3560070712786,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/EPICERIE/3560070712786_PHOTOSITE_20181023_161553_0.jpg?placeholder=1,Café bio équilibré CARREFOUR BIO pas cher au Meilleur Prix,"Acheter votre Café bio équilibré CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:53:22,https://www.carrefour.fr/p/cafe-bio-equilibre-carrefour-bio-3560070712786,19-01-17\n,crf
3,pain-bio-cereales-carrefour-bio-3523680347823,Pain bio céréales CARREFOUR BIO,Carrefour Bio,Accueil > Pains et Pâtisseries > Mon boulanger > Pain,,"Acheter votre Pain bio céréales CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",pain-bio-cereales-carrefour-bio-3523680347823,https://www.carrefour.fr/media/1500x1500/Photosite/PRODUITS_FRAIS_TRANSFORMATION/BOULANGERIE_PATISSERIE/3523680347823_PHOTOSITE_20171021_043559_0.jpg?placeholder=1,Pain bio céréales CARREFOUR BIO pas cher au Meilleur Prix,"Acheter votre Pain bio céréales CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:53:22,https://www.carrefour.fr/p/pain-bio-cereales-carrefour-bio-3523680347823,19-01-17\n,crf
4,croissant-bio-3276550592037,Croissant bio,Carrefour Bio,Accueil > Pains et Pâtisseries > Viennoiseries et brioches > Viennoiseries,,"Ac

In [7]:
orgc_descriptions.describe()

,xid,xtitle,xbrand,xbreadcrumb,xsubtitle,xdescription,xslug,ximg,xmeta_title,xmeta_description,xmeta_keywords,xapi,xweb,xcrawled_at,xurl,xru,distributeur
count,17123,17123,17123,17123,17123,17123,17123,17123,17123,17123,17123,17123,17123,17123,17123,17123,17123
unique,17123,16754,1298,1169,8636,16646,17123,17076,8966,16905,3169,1,4,3998,17123,1,4
top,vin-rose-bio-de-pays-le-canon-du-marechal-3248843070072,Lait bio demi-écrémé LACTEL,Priméal,Accueil > Alimentation bio > Epicerie salée > Conserve & Plat cuisiné bio,,,vin-rose-bio-de-pays-le-canon-du-marechal-3248843070072,,,"Mille et une vertus. Ces plantes ont été cultivées et élaborées pour vous avec le plus grand soin. Elles ont conservé toute leur vitalité. Plantes aux mille vertus, elles ont une de plus, elles sont le",,v1.0,gwz,2019-01-17 19:33:25,https://naturalia.fr/terrine-foie-de-volaille-180g,19-01-17\n,gwz
freq,1,5,587,191,6117,157,1,11,8009,13,13886,17123,8007,15,1,17123,8007


On constate que le "xtitle" il y a 16754 titres distincts<br>
On peut regarder comment sont les "titles" en doublons

In [8]:
unique_titles = orgc_descriptions['xtitle'].unique().tolist()

In [9]:
%%time
liste_duptitles = []
for title in unique_titles:
    count_title = 0
    for row in orgc_descriptions.itertuples():
        if title in row.xtitle:
            count_title += 1
        if count_title == 2:
            liste_duptitles.append(title) 

CPU times: user 10min 16s, sys: 29.8 ms, total: 10min 16s
Wall time: 10min 16s


In [10]:
len(list(set(liste_duptitles)))

968

In [11]:
orgc_descriptions[orgc_descriptions['xtitle'].isin(list(set(liste_duptitles)))].sort_values(['xtitle','xweb'])[['xtitle','xweb']]

,xtitle,xweb
2336,,wbe
2490,Abricots Bergeron CARREFOUR BIO,crf
2492,Abricots Bergeron CARREFOUR BIO,crf
649,Abricots Bruns Moelleux,wbe
1083,Abricots Bruns Moelleux,wbe
2517,Agar-Agar Gélifiant Végétal,wbe
2652,Agar-Agar Gélifiant Végétal,wbe
774,Ail des Ours,wbe
1402,Alfalfa Graines à Germer,wbe
68,Aloe vera bio,crf


In [12]:
# Mise en évidence de produits identiques avec distributeurs différents
df_same_titles = orgc_descriptions[orgc_descriptions['xtitle'].isin(list(set(liste_duptitles)))].sort_values(['xtitle','xweb'])[['xtitle','xweb']]

In [13]:
df_temp = pd.merge(df_same_titles,df_same_titles, how = "inner", on = "xtitle")

In [14]:
df_temp[df_temp['xweb_x'] != df_temp['xweb_y']]

,xtitle,xweb_x,xweb_y
425,Coquillettes Blanches 500g,gwz,ntr
426,Coquillettes Blanches 500g,ntr,gwz
467,Couscous Demi-complet 500g,gwz,ntr
468,Couscous Demi-complet 500g,ntr,gwz
657,Galettes champignons pois chiches,gwz,wbe
658,Galettes champignons pois chiches,wbe,gwz
663,Galettes tomates lentilles corail,gwz,wbe
664,Galettes tomates lentilles corail,wbe,gwz
683,Gluten Pur,gwz,wbe
684,Gluten Pur,wbe,gwz


In [ ]:
ON VOIT QUE A TITRE EGAUX, ON TROUVE DES PRODUITS QUI MATCHENT, CA PEUT SERVIR EN COMPLEMENT DU RESULTAT FINAL

On doit créer un titre plus précis avec xtitle + xbrand + xsubtitle<br>
Mais pour l'instant je ne prends que : xtitle + xbrand, car le "xsubtitle" est trop détaillé pour GREENWEEZ !

In [16]:
orgc_descriptions[orgc_descriptions['xsubtitle'] != '']

,xid,xtitle,xbrand,xbreadcrumb,xsubtitle,xdescription,xslug,ximg,xmeta_title,xmeta_description,xmeta_keywords,xapi,xweb,xcrawled_at,xurl,xru,distributeur
0,vitabio-lot-de-3-boissons-fruit-kid-z-pomme-mangue-acerola-20cl-p44836,Lot de 3 Boissons Fruit Kid'z Pomme Mangue Acérola 20cl,Vitabio,Accueil > Boissons > Jus de fruits bio > Jus multifruits bio,"Délicieux Cocktail 100% pur jus de fruits. Facile à transporter. Parfum jaune: pomme, mangue, acérola. A partir de 3 ans. Délicieux Cocktail 100% pur jus de fruits. Facile à transporter. Parfum jaune: pomme, mangue, acérola. A partir de 3 ans.","Délicieux Cocktail 100% pur jus de fruits. Facile à transporter. Parfum jaune: pomme, mangue, acérola.A partir de 3 ans.Délicieux Cocktail 100% pur jus de fruits. Facile à transporter. Parfum jaune: pomme, mangue, acérola.A partir de 3 ans.",vitabio-lot-de-3-boissons-fruit-kid-z-pomme-mangue-acerola-20cl-p44836,https://static.greenweez.com/images/products/44000/600/kalibio-fruit-kid-z-kalibio-jaune-3x20cl.jpg,,"Lot de 3 Boissons Fruit Kid'z Pomme Mangue Acérola 20cl Vitabio - Jus multifruits bio - Jus de fruits bio - Délicieux Cocktail 100% pur jus de fruits. Facile à transporter. Parfum jaune: pomme, mangue, acérola.A partir de 3 ans.",,v1.0,gwz,2019-01-17 19:09:39,https://www.greenweez.com/vitabio-lot-de-3-boissons-fruit-kid-z-pomme-mangue-acerola-20cl-p44836,19-01-17\n,gwz
1,holle-biscuits-a-l-epeautre-pour-bebe-des-8-mois-p6482,Biscuits à l'épeautre pour bébé - dès 8 mois,Holle,Accueil > Alimentation Bébé Bio > Goûters & biscuits bio bébé > Goûters bio bébé 8 mois,"Des biscuits à l'épeautre pour bébé, à partir de 8 mois. Paquet de 150 grammes. Des biscuits à l'épeautre pour bébé, à partir de 8 mois. Paquet de 150 grammes.","Des biscuits à l'épeautre pour bébé, à partir de 8 mois. Paquet de 150 grammes.Des biscuits à l'épeautre pour bébé, à partir de 8 mois. Paquet de 150 grammes.",holle-biscuits-a-l-epeautre-pour-bebe-des-8-mois-p6482,https://static.greenweez.com/images/products/6000/600/holle-biscuits-a-l-epeautre-pour-bebe-des-8-mois.jpg,,"Biscuits à l'épeautre pour bébé - dès 8 mois Holle - Goûters bio bébé 8 mois - Goûters & biscuits bio bébé - Des biscuits à l'épeautre pour bébé, à partir de 8 mois. Paquet de 150 grammes.",,v1.0,gwz,2019-01-17 19:09:39,https://www.greenweez.com/holle-biscuits-a-l-epeautre-pour-bebe-des-8-mois-p6482,19-01-17\n,gwz
2,hipp-soupe-legumes-verts-petites-pates-p20407,Soupe Légumes Verts & Petites pâtes,HiPP,Accueil > Alimentation Bébé Bio > Soupes bio bébé > Soupes bio bébé 8-12 mois,"Aliments issus de l'Agriculture Biologique, légumes récoltés à maturité - 2 briques de 250 ml - dès 12 mois. Idéal pour le diner : 1 brique cale l'estomac de bébé pour toute la nuit. Hipp, des menus pour bébé gourmets, sains et équilibrés. Aliments issus de l'Agriculture Biologique, légumes récoltés à maturité - 2 briques de 250 ml - dès 12 mois. Idéal pour le diner : 1 brique cale l'estomac de bébé pour toute la nuit. Hipp, des menus pour bébé gourmets, sains et équilibrés.","Aliments issus de l'Agriculture Biologique, légumes récoltés à maturité - 2 briques de 250 ml - dès 12 mois. Idéal pour le diner : 1 brique cale l'estomac de bébé pour toute la nuit. Hipp, des menus pour bébé gourmets, sains et équilibrés.Aliments issus de l'Agriculture Biologique, légumes récoltés à maturité - 2 briques de 250 ml - dès 12 mois. Idéal pour le diner : 1 brique cale l'estomac de bébé pour toute la nuit. Hipp, des menus pour bébé gourmets, sains et équilibrés.",hipp-soupe-legumes-verts-petites-pates-p20407,https://static.greenweez.com/images/products/20000/600/hipp-soupe-legumes-verts-petites-pates.jpg,,"Soupe Légumes Verts & Petites pâtes HiPP - Soupes bio bébé 8-12 mois - Soupes bio bébé - Aliments issus de l'Agriculture Biologique, légumes récoltés à maturité - 2 briques de 250 ml - dès 12 mois. Idéal pour le diner : 1 brique cale l'estomac de bébé pour toute la nuit. Hipp, des menus pour bébé gourmets, sains et équilibrés.",,v1.0,gwz

In [37]:
orgc_descriptions[orgc_descriptions['xsubtitle'] != ''].shape

(11006, 17)

In [38]:
orgc_descriptions[orgc_descriptions['xbrand'] != ''].shape

(16866, 17)

In [ ]:
ATTENTION !!!<br>
On utilise le fichier des stopwords d'origine : "all_stopwords_dedup_for_titles_without_letters_cleansed_0.csv"<br>
et on garde les "d'" en ne lançant pas le code ci-dessous

In [21]:
# def rexexp_suppl(x):
#     resultat = re.sub("(\'|\’)","' ",x)
#     return resultat

# wine_descriptions['description_regexp'] = wine_descriptions['description'].map(rexexp_suppl)
# wine_descriptions = wine_descriptions.drop("description", axis =1)
# wine_descriptions.rename(columns={'description_regexp': 'description'}, inplace=True)

In [22]:
# %%time
# #### On essaye de mettre une majuscule en 1er si présence d'une majuscule 
# # pour la 1ère lettre d'un mot !

# import nltk
# from nltk.tokenize import word_tokenize

# import re

# def capitalize_each_word(x):
#     resultat = ''
# #     pattern = r'^[A-Z][a-z]*(?:_[A-Z][a-z]*)*$'
#     pattern = r'[A-Z][a-z]*'
    
#     description_words = word_tokenize(x)
    
#     for word in description_words:
#         if re.match(pattern, word):
# #             print (word)
#             resultat = resultat + ' ' + word.capitalize()
#         else:
#             resultat = resultat + ' ' + word
  
#     return resultat

Ci-dessous : PAS DANS LE PIPELINE
Je cherche le nombre de mots total dans toutes les descriptions

In [43]:
# %%time
# from functions import fun_data_cleansing as fd

# number_of_words = 0
# for row in orgc_descriptions.itertuples():
#     description_temp = row.xdescription
#     number_temp = len(description_temp.split(' '))
    
#     number_of_words += number_temp
    
# print (number_of_words)

# Il y a 738338 mots avec split(' ') en tout parmi les 16646 descriptions distinctes de produits BIO
# J'ai .....  mots avec "word_tokenize"

# Donc les 100000 mots sont largement atteints afin d'appliquer (cf andrew Ng COURS 5 sur word embeddings)
# du "Transfer learning" pour faire du "word embeddings" 

738338
CPU times: user 67.2 ms, sys: 4.04 ms, total: 71.2 ms
Wall time: 70.1 ms


ATTENTION !!!<br>
On teste en gardant les "'"

In [25]:
# wine_figaro_descriptions['description_regexp'] = wine_figaro_descriptions['description'].map(rexexp_suppl)
# wine_figaro_descriptions = wine_figaro_descriptions.drop("description", axis =1)
# wine_figaro_descriptions.rename(columns={'description_regexp': 'description'}, inplace=True)

Ci-dessous : PAS DANS LE PIPELINE

In [29]:
### ON VEUT TESTER si un titre peut etre découpé : "Domaine Anne Gros Clos-de-Vougeot Le Grand Maupertui"
# On va insérer une ligne dans "wine_descriptions" avec la colonne "title"
# cf http://avis-vin.lefigaro.fr/vins-champagne/bourgogne/cote-de-nuits/bourgogne/d11273-domaine-anne-gros/v11905-domaine-anne-gros-clos-de-vougeot-le-grand-maupertui/vin-rouge


# var_title = "Domaine Anne Gros Clos-de-Vougeot Le Grand Maupertui"
# wine_descriptions = wine_descriptions.append({'slug':'',
#                           'fil_ariane':'',
#                           'title':var_title,
#                           'url':'',
#                           'description':'',
#                           'image_url':'',
#                           'column_one':''}, ignore_index=True)

# wine_descriptions[wine_descriptions["title"] == "Domaine Anne Gros Clos-de-Vougeot Le Grand Maupertui"]

In [30]:
# On va regarder si il peut être découpé ?
# Est-ce des entités déjà trouvées grace au résultat n°2 peuvent découper ce titre ?

In [31]:
# Corpus = toutes les descriptions stocké dans une liste
# corpus = []
# for row in wine_descriptions.itertuples():
#     corpus.append(row.description)

#### step_1_data_cleansing
Ne concerne que les titres

ATTENTION : "d'" est à enlever de la liste des stopwords<br>
187 completement<br>
188 d'<br>
189 d'abord<br>

ATTENTION : or ne doit pas être enlevé ! donc j'enlève "or" de cette liste 
737 what
738 or
739 wasn

In [87]:
# A faire au début, pour utiliser "%autoreload" : 
%load_ext autoreload
# Appl
%autoreload

import pandas as pd
pd.set_option('display.max_colwidth',-1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ATTENTION !!!<br>
On teste en gardant les "'"

Pour le BIO, on se demande quels caractères spéciaux sont présents dans les titres ?

In [54]:
# %%time

# all_special_characters_in_titles = []
# for row in orgc_descriptions.itertuples():
#     all_special_temp = re.sub('[A-Za-z0-9]+', '', unicodedata.normalize('NFD', row.xtitle).encode('ascii', 'ignore').decode('ascii')).strip()
#     if all_special_temp not in all_special_characters_in_titles:
#         all_special_characters_in_titles.append(all_special_temp)

CPU times: user 84.7 ms, sys: 3 µs, total: 84.7 ms
Wall time: 83.7 ms


In [55]:
# all_special_characters_in_titles

['',
 ",    '",
 "'",
 "-  '",
 "-/  '",
 "'  '",
 ',',
 "-      '",
 "%  '",
 "-    '",
 "'   '",
 ", &   '",
 "- '",
 '-',
 "'    '",
 "-   '",
 "-  / '",
 '!',
 '/',
 ", / '",
 ', /',
 ',  /',
 '/ ,',
 '&',
 ', //',
 '//',
 ",  '",
 '%',
 "/  '",
 ',   ,',
 "%   '",
 '.',
 "' /",
 "'   '  '",
 "/,    '",
 "/ '",
 '+',
 "/    '",
 "' '",
 "'  &",
 ",   '",
 "',",
 ':       %',
 ': ***',
 ':    %',
 ':   ***',
 ':  ***',
 ':     %',
 ':  **',
 "&  /     '",
 '- %',
 "'   -",
 '/ &',
 ', ,',
 '% . .',
 ',%',
 '-  &',
 '-%',
 "&   '",
 '%  .',
 "&     '",
 '%   %',
 ',  &  -  &',
 ',  &  -',
 ', ,   ,',
 "'    &",
 "&    '",
 "/ /  '",
 ",     / '",
 "' / '",
 '/   !',
 "-- -  '",
 "- -  '",
 ',  ,',
 "/   '",
 ', -',
 "'     '",
 "&  ' -",
 "'  /",
 ", '   !",
 "/  &  , '   !",
 ':  -   &',
 '+ : -   &',
 ':      &',
 ': -   &',
 "' '    '   '",
 "' //",
 '-  +',
 '- /',
 "'  %",
 "'   %",
 "'    /",
 '-- " "',
 '--',
 ',   &',
 "'/",
 "'    %",
 '-   , %  -',
 ', %  -',
 '- , %  -',
 

In [6]:
orgc_descriptions[(orgc_descriptions['xtitle'].str.contains("/"))]

,xid,xtitle,xbrand,xbreadcrumb,xsubtitle,xdescription,xslug,ximg,xmeta_title,xmeta_description,xmeta_keywords,xapi,xweb,xcrawled_at,xurl,xru,distributeur
88,nettoyant-menager-anti-calcaire-vinaigre-blanc-l-arbre-vert-3450601013638,Nettoyant ménager anti-calcaire/vinaigre blanc L'ARBRE VERT,Arbre Vert,Accueil > Entretien et nettoyage > Produits nettoyants > Cuisine et Salle de bain,,"Acheter votre Nettoyant ménager anti-calcaire/vinaigre blanc L'ARBRE VERT en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",nettoyant-menager-anti-calcaire-vinaigre-blanc-l-arbre-vert-3450601013638,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/DROGUERIE/3450601013638_PHOTOSITE_20180424_162458_0.jpg?placeholder=1,Nettoyant ménager anti-calcaire/vinaigre blanc L'ARBRE VERT pas cher au Meilleur Prix,"Acheter votre Nettoyant ménager anti-calcaire/vinaigre blanc L'ARBRE VERT en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:53:42,https://www.carrefour.fr/p/nettoyant-menager-anti-calcaire-vinaigre-blanc-l-arbre-vert-3450601013638,19-01-17\n,crf
180,creme-masque-visage-nuit-recuperatrice-so-bio-etic-3517360012743,Crème-masque visage nuit/récupératrice SO'BIO ETIC,So'Bio Etic,Accueil > Hygiène et Beauté > Le Visage > Soin du visage > Crèmes hydratantes,,"Acheter votre Crème-masque visage nuit/récupératrice SO'BIO ETIC en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",creme-masque-visage-nuit-recuperatrice-so-bio-etic-3517360012743,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/PARFUMERIE_HYGIENE/3517360012743_PHOTOSITE_20180124_125614_0.jpg?placeholder=1,Crème-masque visage nuit/récupératrice SO'BIO ETIC pas cher au Meilleur Prix,"Acheter votre Crème-masque visage nuit/récupératrice SO'BIO ETIC en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:54:06,https://www.carrefour.fr/p/creme-masque-visage-nuit-recuperatrice-so-bio-etic-3517360012743,19-01-17\n,crf
201,petits-pots-bebe-bio-des-4-6-mois-panais-naturnes-7613036150637,Petits pots bébé bio dès 4/6 mois panais NATURNES,Nestle,Accueil > Le Monde de Bébé > Repas de bébé > Légumes,,"Acheter votre Petits pots bébé bio dès 4/6 mois panais NATURNES en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",petits-pots-bebe-bio-des-4-6-mois-panais-naturnes-7613036150637,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/EPICERIE/7613036150637_PHOTOSITE_20181228_045440_0.jpg?placeholder=1,Petits pots bébé bio dès 4/6 mois panais NATURNES pas cher au Meilleur Prix,"Acheter votre Petits pots bébé bio dès 4/6 mois panais NATURNES en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:54:11,https://www.carrefour.fr/p/petits-pots-bebe-bio-des-4-6-mois-panais-naturnes-7613036150637,19-01-17\n,crf
205,petits-pots-bebe-bio-des-4-6-mois-carotte-naturnes-7613036150767,Petits pots bébé bio dès 4/6 mois carotte NATURNES,Nestle,Accueil > Le Monde de Bébé > Repas de bébé > Légumes,,"Acheter votre Petits pots bébé bio dès 4/6 mois carotte NATURNES en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",petits-pots-bebe-bio-des-4-6-mois-carotte-naturnes-7613036150767,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/EPICERIE/7613036150767_PHOTOSITE_20181228_045426_0.jpg?placeholder=1,Petits pots bébé bio dès 4/6 mois carotte NATURNES pas cher au Meilleur Prix,"Acheter votre Petits pots 

In [96]:
orgc_descriptions[(orgc_descriptions['xtitle'].str.contains("-"))]

,xid,xtitle,xbrand,xbreadcrumb,xsubtitle,xdescription,xslug,ximg,xmeta_title,xmeta_description,xmeta_keywords,xapi,xweb,xcrawled_at,xurl,xru,distributeur
84,nettoyant-menager-multi-surfaces-romarin-l-arbre-vert-3450601026171,Nettoyant ménager multi-surfaces romarin L'ARBRE VERT,Arbre Vert,Accueil > Entretien et nettoyage > Produits nettoyants > Multi-usages,,"Acheter votre Nettoyant ménager multi-surfaces romarin L'ARBRE VERT en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",nettoyant-menager-multi-surfaces-romarin-l-arbre-vert-3450601026171,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/DROGUERIE/3450601026171_PHOTOSITE_20180426_160421_0.jpg?placeholder=1,Nettoyant ménager multi-surfaces romarin L'ARBRE VERT pas cher au Meilleur Prix,"Acheter votre Nettoyant ménager multi-surfaces romarin L'ARBRE VERT en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:53:42,https://www.carrefour.fr/p/nettoyant-menager-multi-surfaces-romarin-l-arbre-vert-3450601026171,19-01-17\n,crf
88,nettoyant-menager-anti-calcaire-vinaigre-blanc-l-arbre-vert-3450601013638,Nettoyant ménager anti-calcaire/vinaigre blanc L'ARBRE VERT,Arbre Vert,Accueil > Entretien et nettoyage > Produits nettoyants > Cuisine et Salle de bain,,"Acheter votre Nettoyant ménager anti-calcaire/vinaigre blanc L'ARBRE VERT en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",nettoyant-menager-anti-calcaire-vinaigre-blanc-l-arbre-vert-3450601013638,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/DROGUERIE/3450601013638_PHOTOSITE_20180424_162458_0.jpg?placeholder=1,Nettoyant ménager anti-calcaire/vinaigre blanc L'ARBRE VERT pas cher au Meilleur Prix,"Acheter votre Nettoyant ménager anti-calcaire/vinaigre blanc L'ARBRE VERT en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:53:42,https://www.carrefour.fr/p/nettoyant-menager-anti-calcaire-vinaigre-blanc-l-arbre-vert-3450601013638,19-01-17\n,crf
90,nettoyant-menager-multi-usages-agrumes-l-arbre-vert-3450601010972,Nettoyant ménager multi-usages agrumes L'ARBRE VERT,Arbre Vert,Accueil > Entretien et nettoyage > Produits nettoyants > Multi-usages,,"Acheter votre Nettoyant ménager multi-usages agrumes L'ARBRE VERT en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",nettoyant-menager-multi-usages-agrumes-l-arbre-vert-3450601010972,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/DROGUERIE/3450601010972_PHOTOSITE_20170825_160558_0.jpg?placeholder=1,Nettoyant ménager multi-usages agrumes L'ARBRE VERT pas cher au Meilleur Prix,"Acheter votre Nettoyant ménager multi-usages agrumes L'ARBRE VERT en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:53:42,https://www.carrefour.fr/p/nettoyant-menager-multi-usages-agrumes-l-arbre-vert-3450601010972,19-01-17\n,crf
111,roll-on-sos-special-hiver-eucalyrhume-bio-so-bio-etic-3517360016413,Roll-on SOS spécial hiver eucalyrhume bio SO'BIO ÉTIC,So'Bio Etic,Accueil > Hygiène et Beauté > Huiles essentielles,,"Acheter votre Roll-on SOS spécial hiver eucalyrhume bio SO'BIO ÉTIC en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",roll-on-sos-special-hiver-eucalyrhume-bio-so-bio-etic-3517360016413,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/PARFUMERIE_HYGIENE/3517360016413_PHOTOSITE_20181112_133655_0.jpg?placeholder=1,Roll-on SOS spécial hiver eucalyrhume bio S

In [97]:
orgc_descriptions[(orgc_descriptions['xtitle'].str.contains(" - "))]

,xid,xtitle,xbrand,xbreadcrumb,xsubtitle,xdescription,xslug,ximg,xmeta_title,xmeta_description,xmeta_keywords,xapi,xweb,xcrawled_at,xurl,xru,distributeur
1447,blanc-de-poulet-bio-25-de-sel-fleury-michon-3095759498014,Blanc de poulet bio - 25% de sel FLEURY MICHON,,Accueil > Charcuterie > Jambons de volaille,,"Acheter votre Blanc de poulet bio - 25% de sel FLEURY MICHON en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",blanc-de-poulet-bio-25-de-sel-fleury-michon-3095759498014,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/P.L.S./3095759498014_PHOTOSITE_20181009_161119_0.jpg?placeholder=1,Blanc de poulet bio - 25% de sel FLEURY MICHON pas cher au Meilleur Prix,"Acheter votre Blanc de poulet bio - 25% de sel FLEURY MICHON en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:59:14,https://www.carrefour.fr/p/blanc-de-poulet-bio-25-de-sel-fleury-michon-3095759498014,19-01-17\n,crf
1708,puree-de-fruits-pomme-fraise-basilic-bio-sans-sucres-ajoutes-yooji-3760234500529,"Purée de fruits pomme, fraise & basilic - Bio & sans sucres ajoutés YOOJI",Yooji,"Accueil > Surgelés > Desserts, viennoiseries et fruits > Fruits et coulis",,"Acheter votre Purée de fruits pomme, fraise & basilic - Bio & sans sucres ajoutés YOOJI en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",puree-de-fruits-pomme-fraise-basilic-bio-sans-sucres-ajoutes-yooji-3760234500529,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/P.L.S./3760234500529_PHOTOSITE_20181213_161827_0.jpg?placeholder=1,"Purée de fruits pomme, fraise & basilic - Bio & sans sucres ajoutés YOOJI pas cher au Meilleur Prix","Acheter votre Purée de fruits pomme, fraise & basilic - Bio & sans sucres ajoutés YOOJI en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 20:00:23,https://www.carrefour.fr/p/puree-de-fruits-pomme-fraise-basilic-bio-sans-sucres-ajoutes-yooji-3760234500529,19-01-17\n,crf
1709,puree-de-fruits-pomme-mangue-verveine-bio-et-sans-sucres-ajoutes-yooji-3760234500536,"Purée de fruits pomme, mangue & verveine - Bio et sans sucres ajoutés YOOJI",Yooji,"Accueil > Surgelés > Desserts, viennoiseries et fruits > Fruits et coulis",,"Acheter votre Purée de fruits pomme, mangue & verveine - Bio et sans sucres ajoutés YOOJI en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",puree-de-fruits-pomme-mangue-verveine-bio-et-sans-sucres-ajoutes-yooji-3760234500536,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/P.L.S./3760234500536_PHOTOSITE_20181213_161617_0.jpg?placeholder=1,"Purée de fruits pomme, mangue & verveine - Bio et sans sucres ajoutés YOOJI pas cher au Meilleur Prix","Acheter votre Purée de fruits pomme, mangue & verveine - Bio et sans sucres ajoutés YOOJI en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 20:00:23,https://www.carrefour.fr/p/puree-de-fruits-pomme-mangue-verveine-bio-et-sans-sucres-ajoutes-yooji-3760234500536,19-01-17\n,crf
2335,puree-de-fruits-pomme-banane-canelle-bio-sans-gluten-et-sans-sucres-ajoutes-yooji-3760234500505,Purée de fruits pomme banane canelle - Bio sans gluten et sans sucres ajoutés YOOJI,Yooji,"Accueil > Surgelés > Desserts, viennoiseries et fruits > Fruits et coulis",,"Acheter votre Purée de fruits pomme banane canelle - Bio sans gluten et sans sucres ajoutés YOOJI en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",puree

In [91]:
# orgc_descriptions[(orgc_descriptions['xtitle'].str.contains("+"))]
# orgc_descriptions[(orgc_descriptions['xtitle'].str.contains('\\'))]

In [92]:
orgc_descriptions[(orgc_descriptions['xtitle'].str.contains('"'))]

,xid,xtitle,xbrand,xbreadcrumb,xsubtitle,xdescription,xslug,ximg,xmeta_title,xmeta_description,xmeta_keywords,xapi,xweb,xcrawled_at,xurl,xru,distributeur
3009,vin-rouge-bio-cotes-du-rhone-vin-gourmand-dauvergne-ranvier-3700689906552,"Vin rouge bio Côtes-du-Rhône ""Vin Gourmand"" DAUVERGNE RANVIER",Refs.Sans Marque,Accueil > Boissons et Cave à vins > Cave à vins > Vins Rouges > Vallée du Rhône,,"Acheter votre Vin rouge bio Côtes-du-Rhône ""Vin Gourmand"" DAUVERGNE RANVIER en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",vin-rouge-bio-cotes-du-rhone-vin-gourmand-dauvergne-ranvier-3700689906552,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/BOISSONS/3700689906552_PHOTOSITE_20180725_153720_0.jpg?placeholder=1,"Vin rouge bio Côtes-du-Rhône ""Vin Gourmand"" DAUVERGNE RANVIER pas cher au Meilleur Prix","Acheter votre Vin rouge bio Côtes-du-Rhône ""Vin Gourmand"" DAUVERGNE RANVIER en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 20:05:35,https://www.carrefour.fr/p/vin-rouge-bio-cotes-du-rhone-vin-gourmand-dauvergne-ranvier-3700689906552,19-01-17\n,crf
692,destination-the-vert-earl-grey-100g-p48643,"Thé vert ""Earl Grey"" 100g",Destination,Accueil > Epicerie > Thés bio et infusions > Thé Vert > Thé vert vrac,"Thé vert ""Earl Grey"" : Thé délicat et légèrement corsé, associé à la fraîcheur de bergamote d'Italie. Aromatisé à la bergamote biologique -Tasse fruitée, fraîche. A déguster à tout moment de la journée sans modération. Sachet de 100g. Thé vert ""Earl Grey"" : Thé délicat et légèrement corsé, associé à la fraîcheur de bergamote d'Italie. Aromatisé à la bergamote biologique -Tasse fruitée, fraîche. A déguster à tout moment de la journée sans modération. Sachet de 100g.","Thé vert ""Earl Grey"" : Thé délicat et légèrement corsé, associé à la fraîcheur de bergamote d'Italie. Aromatisé à la bergamote biologique -Tasse fruitée, fraîche. A déguster à tout moment de la journée sans modération. Sachet de 100g.Thé vert ""Earl Grey"" : Thé délicat et légèrement corsé, associé à la fraîcheur de bergamote d'Italie. Aromatisé à la bergamote biologique -Tasse fruitée, fraîche. A déguster à tout moment de la journée sans modération. Sachet de 100g.",destination-the-vert-earl-grey-100g-p48643,https://static.greenweez.com/images/products/48000/600/destination-the-vert-earl-grey-100g.jpg,,Thé vert,,v1.0,gwz,2019-01-17 19:12:46,https://www.greenweez.com/destination-the-vert-earl-grey-100g-p48643,19-01-17\n,gwz
817,destination-the-noir-bio-earl-grey-ceylan-100g-p19843,"Thé Noir BIO ""Earl Grey Ceylan"" 100g",Destination,Accueil > Epicerie > Thés bio et infusions > Thé Noir,"Doux et fruité, le thé noir BIO Earl Grey vous offre une tasse pétillante de fraîcheur. Sachet de 100g. Doux et fruité, le thé noir BIO Earl Grey vous offre une tasse pétillante de fraîcheur. Sachet de 100g.","Doux et fruité, le thé noir BIO Earl Grey vous offre une tasse pétillante de fraîcheur. Sachet de 100g.Doux et fruité, le thé noir BIO Earl Grey vous offre une tasse pétillante de fraîcheur. Sachet de 100g.",destination-the-noir-bio-earl-grey-ceylan-100g-p19843,https://static.greenweez.com/images/products/19000/600/destination-the-noir-bio-earl-grey-ceylan-100g.jpg,,Thé Noir BIO,,v1.0,gwz,2019-01-17 19:13:20,https://www.greenweez.com/destination-the-noir-bio-earl-grey-ceylan-100g-p19843,19-01-17\n,gwz
972,lima-mu-sachets-epice-et-doux-10x1-5g-p18088,"Mu sachets ""épicé et doux"" 10x1.5g",Lima,"Accueil > Epicerie > Thés bio et infusions > Tisanes, Infusions > Tisanes de fruits","Tisane Mu Sachets ""épicé et doux"" de la marque Lima. Contient 15 plantes et racines et du ginseng. 15g. Tisane Mu Sachets ""épicé et doux"" de la marque Lima. Contient 15 plantes et racines et du ginseng. 15g.","Tisane Mu Sachets ""épicé et doux"" de la marque Lima. Contient 15

In [93]:
orgc_descriptions[(orgc_descriptions['xtitle'].str.contains(" ' "))]

,xid,xtitle,xbrand,xbreadcrumb,xsubtitle,xdescription,xslug,ximg,xmeta_title,xmeta_description,xmeta_keywords,xapi,xweb,xcrawled_at,xurl,xru,distributeur


In [94]:
orgc_descriptions[(orgc_descriptions['xtitle'].str.contains("''"))]

,xid,xtitle,xbrand,xbreadcrumb,xsubtitle,xdescription,xslug,ximg,xmeta_title,xmeta_description,xmeta_keywords,xapi,xweb,xcrawled_at,xurl,xru,distributeur


In [95]:
orgc_descriptions[(orgc_descriptions['xtitle'].str.contains(re.compile(r'\s+')))]

,xid,xtitle,xbrand,xbreadcrumb,xsubtitle,xdescription,xslug,ximg,xmeta_title,xmeta_description,xmeta_keywords,xapi,xweb,xcrawled_at,xurl,xru,distributeur
0,fromage-bio-frais-carrefour-bio-3245413442420,Fromage bio frais CARREFOUR BIO,Carrefour Bio,"Accueil > Crémerie > Yaourts, desserts et spécialités végétales > Yaourts Nature et Fromages blancs",,"Acheter votre Fromage bio frais CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",fromage-bio-frais-carrefour-bio-3245413442420,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/P.L.S./3245413442420_PHOTOSITE_20160922_080810_0.jpg?placeholder=1,Fromage bio frais CARREFOUR BIO pas cher au Meilleur Prix,"Acheter votre Fromage bio frais CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:53:16,https://www.carrefour.fr/p/fromage-bio-frais-carrefour-bio-3245413442420,19-01-17\n,crf
1,yaourts-bio-nature-carrefour-bio-3245413442338,Yaourts bio nature CARREFOUR BIO,Carrefour Bio,"Accueil > Crémerie > Yaourts, desserts et spécialités végétales > Yaourts Nature et Fromages blancs",,"Acheter votre Yaourts bio nature CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",yaourts-bio-nature-carrefour-bio-3245413442338,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/P.L.S./3245413442338_PHOTOSITE_20161005_084045_0.jpg?placeholder=1,Yaourts bio nature CARREFOUR BIO pas cher au Meilleur Prix,"Acheter votre Yaourts bio nature CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:53:22,https://www.carrefour.fr/p/yaourts-bio-nature-carrefour-bio-3245413442338,19-01-17\n,crf
2,cafe-bio-equilibre-carrefour-bio-3560070712786,Café bio équilibré CARREFOUR BIO,Carrefour Bio,Accueil > Epicerie Sucrée > Boissons du matin et du soir > Cafés moulus et en grains,,"Acheter votre Café bio équilibré CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",cafe-bio-equilibre-carrefour-bio-3560070712786,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/EPICERIE/3560070712786_PHOTOSITE_20181023_161553_0.jpg?placeholder=1,Café bio équilibré CARREFOUR BIO pas cher au Meilleur Prix,"Acheter votre Café bio équilibré CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:53:22,https://www.carrefour.fr/p/cafe-bio-equilibre-carrefour-bio-3560070712786,19-01-17\n,crf
3,pain-bio-cereales-carrefour-bio-3523680347823,Pain bio céréales CARREFOUR BIO,Carrefour Bio,Accueil > Pains et Pâtisseries > Mon boulanger > Pain,,"Acheter votre Pain bio céréales CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",pain-bio-cereales-carrefour-bio-3523680347823,https://www.carrefour.fr/media/1500x1500/Photosite/PRODUITS_FRAIS_TRANSFORMATION/BOULANGERIE_PATISSERIE/3523680347823_PHOTOSITE_20171021_043559_0.jpg?placeholder=1,Pain bio céréales CARREFOUR BIO pas cher au Meilleur Prix,"Acheter votre Pain bio céréales CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",,v1.0,crf,2019-01-17 19:53:22,https://www.carrefour.fr/p/pain-bio-cereales-carrefour-bio-3523680347823,19-01-17\n,crf
4,croissant-bio-3276550592037,Croissant bio,Carrefour Bio,Accueil > Pains et Pâtisseries > Viennoiseries et brioches > Viennoiseries,,"Ac

CONCLUSION pour nettoyer les titres :
- pourquoi le '"' est remplacé par le '' pour le vin ?
- car pour le BIO, il est intéressant de garder ces " ", qui permettent de splitter les titres
afin d'extraire une entité 
- On va essayer de splitter les titres avec :
 les mots à l'intérieur des '"' 
 les mots séparés par ' - ' ou ' : '
 les mots en majuscule
Ensuite il faudra chercher les entités grace au Chunking
dans les parties splittées.

In [17]:
# import re
# # title = "Fromage bio frais CARREFOUR BIO"
# title = "Fromage bio frais carrefour bio"
# pattern_upper = re.compile('[A-Z]+ [A-Z]+')
# upper_extracted = pattern_upper.findall(title)
# upper_extracted[0]

In [28]:
# import re
# # title = 'Fromage "Bio Frais" CARREFOUR BIO'
# title = 'Fromage "Bio frais INCROYABLE" CARREFOUR BIO'
# # pattern_guillemets = re.compile('"[A-za-z]+ [A-za-z]+"')
# pattern_guillemets = r'(?<=\").+?(?=\")'
# # re.findall(pattern_guillemets,title)
# [each.replace('"','') for each in re.findall(pattern_guillemets,title)]

['Bio frais INCROYABLE']

In [18]:
from functions import fun_data_cleansing as fd

orgc_descriptions["title_new"] = orgc_descriptions['xtitle'].map(fd.cleansing_titles)

# Suppression des stopwords
orgc_descriptions["title_without_stopwords"] = orgc_descriptions['title_new'].map(fd.remove_stopwords)

# Ajouter un split en mettant de coté les mots en MAJUSCULES
orgc_descriptions["title_splitted_uppercase"] = orgc_descriptions["title_without_stopwords"].map(fd.split_words_uppercase)
# Ajouter un split en mettant de coté les mots compris entre deux guillemets
orgc_descriptions["title_splitted_guillemets"] = orgc_descriptions["title_without_stopwords"].map(fd.split_words_guillemets)

In [20]:
# Création d'une colonne des titres splittés par les séparateurs : {' , ';' : ',' - '}
orgc_descriptions["title_splitted"] = orgc_descriptions["title_without_stopwords"].map(fd.split_title)

CI-DESSOUS, PARTIE A REVOIR : JE SAIS PAS SI J'UTILISE, CE CODE SERT A SPLITTER les titres SUIVANT 
- entités entre majuscules
- et les entités entre guillemets

In [21]:
# orgc_descriptions["title_splitted_uppercase_guillemets"] = orgc_descriptions.apply(fd.split_title_uppercase_guillemets, axis = 1)
# orgc_descriptions["title_splitted_guillemets"] = orgc_descriptions.apply(fd.split_title_guillemets, axis = 1)
# PARTIE A REVOIR 

# Extraction de l'année = millésime présent dans les titres SERT SEULEMENT AU VIN
# MEME SI IL Y A DU VIN dans les produits BIO, je m'en sers pas 
# EST CE QUE C'EST POSSIBLE DE S'EN SERVIR ?
# orgc_descriptions['year_in_title'] = orgc_descriptions["title_without_stopwords"].map(fd.year_in_title)

In [48]:
orgc_descriptions[orgc_descriptions["xtitle"].str.contains('"')]

,xid,xtitle,xbrand,xbreadcrumb,xsubtitle,xdescription,xslug,ximg,xmeta_title,xmeta_description,...,xcrawled_at,xurl,xru,distributeur,title_new,title_without_stopwords,title_splitted_uppercase,title_splitted_guillemets,title_splitted,year_in_title
3009,vin-rouge-bio-cotes-du-rhone-vin-gourmand-dauvergne-ranvier-3700689906552,"Vin rouge bio Côtes-du-Rhône ""Vin Gourmand"" DAUVERGNE RANVIER",Refs.Sans Marque,Accueil > Boissons et Cave à vins > Cave à vins > Vins Rouges > Vallée du Rhône,,"Acheter votre Vin rouge bio Côtes-du-Rhône ""Vin Gourmand"" DAUVERGNE RANVIER en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",vin-rouge-bio-cotes-du-rhone-vin-gourmand-dauvergne-ranvier-3700689906552,https://www.carrefour.fr/media/1500x1500/Photosite/PGC/BOISSONS/3700689906552_PHOTOSITE_20180725_153720_0.jpg?placeholder=1,"Vin rouge bio Côtes-du-Rhône ""Vin Gourmand"" DAUVERGNE RANVIER pas cher au Meilleur Prix","Acheter votre Vin rouge bio Côtes-du-Rhône ""Vin Gourmand"" DAUVERGNE RANVIER en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",...,2019-01-17 20:05:35,https://www.carrefour.fr/p/vin-rouge-bio-cotes-du-rhone-vin-gourmand-dauvergne-ranvier-3700689906552,19-01-17\n,crf,"Vin rouge bio Cotes-du-Rhone ""Vin Gourmand"" DAUVERGNE RANVIER","Vin rouge bio Cotes-du-Rhone ""Vin Gourmand"" DAUVERGNE RANVIER",[DAUVERGNE RANVIER],[Vin Gourmand],"[Vin rouge bio Cotes-du-Rhone ""Vin Gourmand"" ]",[]
692,destination-the-vert-earl-grey-100g-p48643,"Thé vert ""Earl Grey"" 100g",Destination,Accueil > Epicerie > Thés bio et infusions > Thé Vert > Thé vert vrac,"Thé vert ""Earl Grey"" : Thé délicat et légèrement corsé, associé à la fraîcheur de bergamote d'Italie. Aromatisé à la bergamote biologique -Tasse fruitée, fraîche. A déguster à tout moment de la journée sans modération. Sachet de 100g. Thé vert ""Earl Grey"" : Thé délicat et légèrement corsé, associé à la fraîcheur de bergamote d'Italie. Aromatisé à la bergamote biologique -Tasse fruitée, fraîche. A déguster à tout moment de la journée sans modération. Sachet de 100g.","Thé vert ""Earl Grey"" : Thé délicat et légèrement corsé, associé à la fraîcheur de bergamote d'Italie. Aromatisé à la bergamote biologique -Tasse fruitée, fraîche. A déguster à tout moment de la journée sans modération. Sachet de 100g.Thé vert ""Earl Grey"" : Thé délicat et légèrement corsé, associé à la fraîcheur de bergamote d'Italie. Aromatisé à la bergamote biologique -Tasse fruitée, fraîche. A déguster à tout moment de la journée sans modération. Sachet de 100g.",destination-the-vert-earl-grey-100g-p48643,https://static.greenweez.com/images/products/48000/600/destination-the-vert-earl-grey-100g.jpg,,Thé vert,...,2019-01-17 19:12:46,https://www.greenweez.com/destination-the-vert-earl-grey-100g-p48643,19-01-17\n,gwz,"The vert ""Earl Grey"" 100g","The vert ""Earl Grey"" 100g",[],[Earl Grey],"[The vert , 100g]",[]
817,destination-the-noir-bio-earl-grey-ceylan-100g-p19843,"Thé Noir BIO ""Earl Grey Ceylan"" 100g",Destination,Accueil > Epicerie > Thés bio et infusions > Thé Noir,"Doux et fruité, le thé noir BIO Earl Grey vous offre une tasse pétillante de fraîcheur. Sachet de 100g. Doux et fruité, le thé noir BIO Earl Grey vous offre une tasse pétillante de fraîcheur. Sachet de 100g.","Doux et fruité, le thé noir BIO Earl Grey vous offre une tasse pétillante de fraîcheur. Sachet de 100g.Doux et fruité, le thé noir BIO Earl Grey vous offre une tasse pétillante de fraîcheur. Sachet de 100g.",destination-the-noir-bio-earl-grey-ceylan-100g-p19843,https://static.greenweez.com/images/products/19000/600/destination-the-noir-bio-earl-grey-ceylan-100g.jpg,,Thé Noir BIO,...,2019-01-17 19:13:20,https://www.greenweez.com/destination-the-noir-bio-earl-grey-ceylan-100g-p19843,19-01-17\n,gwz,"The Noir BIO ""Earl Grey Ceylan"" 100g","The Noir BIO ""Earl Grey

In [22]:
# Création du dataframe "title_et_ngrams"
# title_et_ngrams = orgc_descriptions[['xtitle','xbrand','xbreadcrumb','xsubtitle','xdescription']]
# Ou 
title_et_ngrams = orgc_descriptions[['xslug','distributeur','title_without_stopwords','xdescription']]

# On met en minuscules la colonne ''title_without_stopwords'
title_et_ngrams["title_temp"] = title_et_ngrams['title_without_stopwords'].map(lambda x: (x.strip()).lower())
title_et_ngrams = title_et_ngrams.drop("title_without_stopwords", axis =1)
title_et_ngrams.rename(columns={'title_temp': 'title_without_stopwords'}, inplace=True)

title_et_ngrams.rename(columns={'xslug': 'slug'}, inplace=True)

# Suppression de séparateurs présents dans les titres de type : '\s*[,:]\s*' 
title_et_ngrams['title_temp'] = title_et_ngrams['title_without_stopwords'].map(fd.remove_separateurs)

title_et_ngrams['title_without_stopwords_new'] = title_et_ngrams['title_temp'].str.replace(' - ',' ')
title_et_ngrams = title_et_ngrams.drop("title_temp", axis =1)

/home/hapax94/.pyenv/versions/my_env_new/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### step_1_ngrams_frequences
Ne concerne que les titres

In [23]:
from functions import fun_data_cleansing as fd

# Suppression de l'année dans les titres
title_et_ngrams["title_without_year"] = title_et_ngrams['title_without_stopwords'].map(fd.remove_year)

title_et_ngrams['title_temp'] = title_et_ngrams['title_without_year'].map(fd.remove_separateurs)

title_et_ngrams['title_without_year_for_words_title'] = title_et_ngrams['title_temp'].str.replace(' - ',' ')

title_et_ngrams = title_et_ngrams.drop("title_temp", axis =1)

 # 10/11/2018 : problème avec les "d'", extrait : "d'" et "''"
# title_et_ngrams['words_title'] = title_et_ngrams['title_without_year_for_words_title'].map(fd.word_tokenize)
title_et_ngrams['words_title'] = title_et_ngrams['title_without_year_for_words_title'].map(lambda x: x.split(' '))

# Sert surtout pour enlever les "' " qui peuvent être présents
title_et_ngrams['words_title_cleansed'] =  title_et_ngrams['words_title'].map(fd.cleansing_words_after_split)

In [24]:
title_et_ngrams.head()

,slug,distributeur,xdescription,title_without_stopwords,title_without_stopwords_new,title_without_year,title_without_year_for_words_title,words_title,words_title_cleansed
0,fromage-bio-frais-carrefour-bio-3245413442420,crf,"Acheter votre Fromage bio frais CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",fromage bio frais carrefour bio,fromage bio frais carrefour bio,fromage bio frais carrefour bio,fromage bio frais carrefour bio,"[fromage, bio, frais, carrefour, bio]","[fromage, bio, frais, carrefour, bio]"
1,yaourts-bio-nature-carrefour-bio-3245413442338,crf,"Acheter votre Yaourts bio nature CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",yaourts bio nature carrefour bio,yaourts bio nature carrefour bio,yaourts bio nature carrefour bio,yaourts bio nature carrefour bio,"[yaourts, bio, nature, carrefour, bio]","[yaourts, bio, nature, carrefour, bio]"
2,cafe-bio-equilibre-carrefour-bio-3560070712786,crf,"Acheter votre Café bio équilibré CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",cafe bio equilibre carrefour bio,cafe bio equilibre carrefour bio,cafe bio equilibre carrefour bio,cafe bio equilibre carrefour bio,"[cafe, bio, equilibre, carrefour, bio]","[cafe, bio, equilibre, carrefour, bio]"
3,pain-bio-cereales-carrefour-bio-3523680347823,crf,"Acheter votre Pain bio céréales CARREFOUR BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",pain bio cereales carrefour bio,pain bio cereales carrefour bio,pain bio cereales carrefour bio,pain bio cereales carrefour bio,"[pain, bio, cereales, carrefour, bio]","[pain, bio, cereales, carrefour, bio]"
4,croissant-bio-3276550592037,crf,"Acheter votre Croissant bio en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",croissant bio,croissant bio,croissant bio,croissant bio,"[croissant, bio]","[croissant, bio]"


In [26]:
# title_et_ngrams[title_et_ngrams['title_without_year'].str.contains("' ")][['title_new','title_without_stopwords','title_splitted','year_in_title','title_without_stopwords_new','title_without_year','title_without_year_for_words_title','words_title','words_title_cleansed']]

Ci-dessous : PAS DANS LE PIPELINE<br>
On crée un .csv pour le donner à R

In [421]:
path_and_file = "/home/hapax94/Documents/vincent/R/minhash_lsh/data/bio_before_lsh.csv"

title_et_ngrams[['distributeur','slug','title_without_stopwords_new']].to_csv(path_and_file,index= False)

In [29]:
list(title_et_ngrams)

['slug',
 'distributeur',
 'xdescription',
 'title_without_stopwords',
 'title_without_stopwords_new',
 'title_without_year',
 'title_without_year_for_words_title',
 'words_title',
 'words_title_cleansed']

In [30]:
title_et_ngrams[title_et_ngrams['title_without_stopwords'].str.contains(" d'")]

,slug,distributeur,xdescription,title_without_stopwords,title_without_stopwords_new,title_without_year,title_without_year_for_words_title,words_title,words_title_cleansed
82,tapenade-d-olives-vertes-divin-bio-3760273430139,crf,"Acheter votre Tapenade d'olives vertes Divin Bio en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",tapenade d'olives vertes divin bio,tapenade d'olives vertes divin bio,tapenade d'olives vertes divin bio,tapenade d'olives vertes divin bio,"[tapenade, d'olives, vertes, divin, bio]","[tapenade, d'olives, vertes, divin, bio]"
95,deodorant-pierre-d-alun-fleur-d-alun-biopha-3286010012500,crf,"Acheter votre Déodorant pierre d'alun fleur d'alun BIOPHA en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",deodorant pierre d'alun fleur d'alun biopha,deodorant pierre d'alun fleur d'alun biopha,deodorant pierre d'alun fleur d'alun biopha,deodorant pierre d'alun fleur d'alun biopha,"[deodorant, pierre, d'alun, fleur, d'alun, biopha]","[deodorant, pierre, d'alun, fleur, d'alun, biopha]"
97,tapenade-d-olives-noires-divin-bio-3760273430122,crf,"Acheter votre Tapenade d'olives noires DIVIN BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",tapenade d'olives noires divin bio,tapenade d'olives noires divin bio,tapenade d'olives noires divin bio,tapenade d'olives noires divin bio,"[tapenade, d'olives, noires, divin, bio]","[tapenade, d'olives, noires, divin, bio]"
121,huile-essentielle-d-orange-biologique-relaxante-so-bio-etic-3478820086424,crf,"Acheter votre Huile essentielle d'orange biologique relaxante SO'BIO ETIC en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",huile essentielle d'orange biologique relaxante so'bio etic,huile essentielle d'orange biologique relaxante so'bio etic,huile essentielle d'orange biologique relaxante so'bio etic,huile essentielle d'orange biologique relaxante so'bio etic,"[huile, essentielle, d'orange, biologique, relaxante, so'bio, etic]","[huile, essentielle, d'orange, biologique, relaxante, so'bio, etic]"
135,huile-essentielle-d-eucalyptus-radiata-bio-so-bio-etic-3517360000757,crf,"Acheter votre Huile essentielle d'eucalyptus radiata bio SO'BIO ÉTIC en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",huile essentielle d'eucalyptus radiata bio so'bio etic,huile essentielle d'eucalyptus radiata bio so'bio etic,huile essentielle d'eucalyptus radiata bio so'bio etic,huile essentielle d'eucalyptus radiata bio so'bio etic,"[huile, essentielle, d'eucalyptus, radiata, bio, so'bio, etic]","[huile, essentielle, d'eucalyptus, radiata, bio, so'bio, etic]"
154,diffuseur-prise-d-huiles-essentiels-chaleur-douce-so-bio-3517360013436,crf,"Acheter votre Diffuseur prise d'huiles essentiels chaleur douce SO'BIO en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.",diffuseur prise d'huiles essentiels chaleur douce so'bio,diffuseur prise d'huiles essentiels chaleur douce so'bio,diffuseur prise d'huiles essentiels chaleur douce so'bio,diffuseur prise d'huiles essentiels chaleur douce so'bio,"[diffuseur, prise, d'huiles, essentiels, chaleur, douce, so'bio]","[diffuseur, prise, d'huiles, essentiels, chaleur, douce, so'bio]"
207,plat-bebe-bio-des-6-mois-courges-saute-d-agneau-good-gout-3770002327050,crf,"Acheter votre Plat bébé bio dès 6 mois, courges/sauté d'agneau GOOD GOÛT en ligne au même prix qu'en magasin, profitez de nos promotions et faites vous livrer ou venez retirer vos courses rapidement dans votre magasin.","plat bebe bio 6 mois, courge

In [422]:
## Création des dictionnaires de 1-grams, 2-grams, 4-grams et 5-grams
# grâce à la colonne 'words_title' du dataframe "title_et_ngrams"

In [ ]:
# Parser
# A parser identifies sentences, words and word types in a string of text. 
# This involves tokenization (distinguishing between abbreviations and sentence breaks), 
# part-of-speech tagging (annotating words with their type, e.g., is
# can a NOUN or a VERB?) and chunking (grouping consecutive words that belong together).
# Parsing can be used to answer questions such as who did what and why and is useful in a wide range 
# of text mining applications. The pattern.en parser uses a lexicon of a 100,000 known words and their part-of-speech
# tag, along with rules for unknown words based on word suffix (e.g., -ly = ADVERB) and context (surrounding words).
# This approach is fast but not always accurate, since many words are ambiguous and hard to capture
# with simple rules. The overall accuracy is about 95% (95.8% on WSJ portions 22-24). 
# It is lower for informal language use (e.g., chat lang

In [31]:
print("\n On traite le dictionnaire des 1-grams !")

dict_all_1_grams = dict()

# 16/02/2018 : on utilise "words_title_cleansed" à la place de "words_title"

for row in title_et_ngrams.itertuples():
    flag_update_dict = False
    
#     ngram_presents = row.words_title
    ngram_presents = row.words_title_cleansed   
    
    for i in range(len(ngram_presents)):
          
        if ngram_presents[i] not in dict_all_1_grams :
            dict_all_1_grams[ngram_presents[i]] = 0
            
            if ngram_presents[i] in row.title_without_year_for_words_title:
                dict_all_1_grams[ngram_presents[i]] = 1
                flag_update_dict = True

        # Si on traite un nouveau "title" de vin
        # et que le ngram en question est déjà dans le dico,
        # On peut incrémenter le compteur 
        if ngram_presents[i] in dict_all_1_grams and flag_update_dict == False:
            if ngram_presents[i] in row.title_without_year_for_words_title:
                 dict_all_1_grams[ngram_presents[i]] = dict_all_1_grams[ngram_presents[i]] + 1

print("\n On traite le dictionnaire des 2-grams !")
dict_all_2_grams = dict()

for row in title_et_ngrams.itertuples():
    flag_update_dict = False
    
#     if find_ngrams(row.words_title,2) != []:
#         ngram_presents = find_ngrams(row.words_title,2)
    if fc.find_ngrams(row.words_title_cleansed,2) != []:
        ngram_presents = fc.find_ngrams(row.words_title_cleansed,2)
        
    for i in range(len(ngram_presents)):
          
        if ngram_presents[i] not in dict_all_2_grams :
            dict_all_2_grams[ngram_presents[i]] = 0 
            
            if ngram_presents[i] in row.title_without_year_for_words_title:
                dict_all_2_grams[ngram_presents[i]] = 1
                flag_update_dict = True

        # Si on traite un nouveau "title" de vin
        # et que le ngram en question est déjà dans le dico,
        # On peut incrémenter le compteur 
        if ngram_presents[i] in dict_all_2_grams and flag_update_dict == False:
            if ngram_presents[i] in row.title_without_year_for_words_title:
                 dict_all_2_grams[ngram_presents[i]] = dict_all_2_grams[ngram_presents[i]] + 1


print("\n On traite le dictionnaire des 3-grams !")
dict_all_3_grams = dict()

for row in title_et_ngrams.itertuples(): 
    flag_update_dict = False
    
#     if find_ngrams(row.words_title,3) != []:
#         ngram_presents = find_ngrams(row.words_title,3)
    if fc.find_ngrams(row.words_title_cleansed,3) != []:
        ngram_presents = fc.find_ngrams(row.words_title_cleansed,3)

    for i in range(len(ngram_presents)):
          
        if ngram_presents[i] not in dict_all_3_grams :
            dict_all_3_grams[ngram_presents[i]] = 0 
            
            if ngram_presents[i] in row.title_without_year_for_words_title:
                dict_all_3_grams[ngram_presents[i]] = 1
                flag_update_dict = True

        # Si on traite un nouveau "title" de vin
        # et que le ngram en question est déjà dans le dico,
        # On peut incrémenter le compteur 
        if ngram_presents[i] in dict_all_3_grams and flag_update_dict == False:
            if ngram_presents[i] in row.title_without_year_for_words_title:
                 dict_all_3_grams[ngram_presents[i]] = dict_all_3_grams[ngram_presents[i]] + 1

print("\n On traite le dictionnaire des 4-grams !")                
dict_all_4_grams = dict()

for row in title_et_ngrams.itertuples():
    flag_update_dict = False
    
    
#     if find_ngrams(row.words_title,4) != []:
#         ngram_presents = find_ngrams(row.words_title,4)
    if fc.find_ngrams(row.words_title_cleansed,4) != []:
        ngram_presents = fc.find_ngrams(row.words_title_cleansed,4)

    for i in range(len(ngram_presents)):
          
        if ngram_presents[i] not in dict_all_4_grams :
            dict_all_4_grams[ngram_presents[i]] = 0 
            
            if ngram_presents[i] in row.title_without_year_for_words_title:
                dict_all_4_grams[ngram_presents[i]] = 1
                flag_update_dict = True

        # Si on traite un nouveau "title" de vin
        # et que le ngram en question est déjà dans le dico,
        # On peut incrémenter le compteur 
        if ngram_presents[i] in dict_all_4_grams and flag_update_dict == False:
            if ngram_presents[i] in row.title_without_year_for_words_title:
                 dict_all_4_grams[ngram_presents[i]] = dict_all_4_grams[ngram_presents[i]] + 1
    
print("\n On traite le dictionnaire des 5-grams !")
dict_all_5_grams = dict()

for row in title_et_ngrams.itertuples():
    flag_update_dict = False
    
#     if find_ngrams(row.words_title,5) != []:
#         ngram_presents = find_ngrams(row.words_title,5)
    if fc.find_ngrams(row.words_title_cleansed,5) != []:
        ngram_presents = fc.find_ngrams(row.words_title_cleansed,5)


    for i in range(len(ngram_presents)):
          
        if ngram_presents[i] not in dict_all_5_grams :
            dict_all_5_grams[ngram_presents[i]] = 0 
            
            if ngram_presents[i] in row.title_without_year_for_words_title:
                dict_all_5_grams[ngram_presents[i]] = 1
                flag_update_dict = True

        # Si on traite un nouveau "title" de vin
        # et que le ngram en question est déjà dans le dico,
        # On peut incrémenter le compteur 
        if ngram_presents[i] in dict_all_5_grams and flag_update_dict == False:
            if ngram_presents[i] in row.title_without_year_for_words_title:
                 dict_all_5_grams[ngram_presents[i]] = dict_all_5_grams[ngram_presents[i]] + 1

del ngram_presents, row, i


 On traite le dictionnaire des 1-grams !

 On traite le dictionnaire des 2-grams !

 On traite le dictionnaire des 3-grams !

 On traite le dictionnaire des 4-grams !

 On traite le dictionnaire des 5-grams !


In [426]:
## Nouvelle étape : dictionnaires avec seulement clés ayant des comptages > 1

In [32]:
dict_all_1_grams_to_update = dict()
for key in dict_all_1_grams.keys():
    if dict_all_1_grams[key] > 1: 
        dict_all_1_grams_to_update[key] = dict_all_1_grams[key] 

dict_all_2_grams_to_update = dict()
for key in dict_all_2_grams.keys():
    if dict_all_2_grams[key] > 1: 
        dict_all_2_grams_to_update[key] = dict_all_2_grams[key] 
        
dict_all_3_grams_to_update = dict()
for key in dict_all_3_grams.keys():
    if dict_all_3_grams[key] > 1: 
        dict_all_3_grams_to_update[key] = dict_all_3_grams[key] 

dict_all_4_grams_to_update = dict()
for key in dict_all_4_grams.keys():
    if dict_all_4_grams[key] > 1: 
        dict_all_4_grams_to_update[key] = dict_all_4_grams[key] 

dict_all_5_grams_to_update = dict()
for key in dict_all_5_grams.keys():
    if dict_all_5_grams[key] > 1: 
        dict_all_5_grams_to_update[key] = dict_all_5_grams[key] 

In [35]:
import operator

df_all_1_grams = pd.DataFrame(sorted(dict_all_1_grams_to_update.items(),key = operator.itemgetter(1),reverse = True),columns = ['un_grams','fréquence'])
df_all_2_grams = pd.DataFrame(sorted(dict_all_2_grams_to_update.items(),key = operator.itemgetter(1),reverse = True),columns = ['deux_grams','frequence'])
df_all_3_grams = pd.DataFrame(sorted(dict_all_3_grams_to_update.items(),key = operator.itemgetter(1),reverse = True),columns = ['trois_grams','frequence'])
df_all_4_grams = pd.DataFrame(sorted(dict_all_4_grams_to_update.items(),key = operator.itemgetter(1),reverse = True),columns = ['quatre_grams','frequence'])

In [36]:
# df_all_2_grams[df_all_2_grams['deux_grams'].str.contains("' ")].head()
# df_all_3_grams[df_all_3_grams['trois_grams'].str.contains("' ")].head()
# df_all_4_grams[df_all_4_grams['quatre_grams'].str.contains("d' ")].head()

#### step_2_extract_xphrases_using_pattern
Ne concerne que les descriptions

In [37]:
df_descriptions_all_retailers = title_et_ngrams[['xdescription']]

#### CODE précédent !

#### On ajoute les entités qui ne sont pas présentes dans les descriptions "all_retailers"
#### mais qui  sont présentes dans les descriptions provenant du "figaro" ! 

In [38]:
from pattern.fr import parsetree

def X_phrases_extraction(x):
    resultat = []
    s = parsetree(x)
    for sentence in s:
        for chunk in sentence.chunks:
#             print (chunk.words)
            resultat.append([chunk.type, ' '.join([w.string for w in chunk.words])])
    return resultat

ATTENTION : Faire plutôt pour extraire aussi toutes les entités sans leurs TAGS !

In [39]:
%%time
X_phrases_descriptions = []
liste_entities_already_extracted = []

for each_text in list(df_descriptions_all_retailers['xdescription']):
    if each_text != "" and each_text != '#':
#         print ("texte vide ! avec:", each_text)
        couples_tag_entity = X_phrases_extraction(each_text)
#         print (couple_tag_entity)
        if couples_tag_entity != []:
#             print (couple_tag_entity)
            for each in couples_tag_entity:
#                 if each not in X_phrases_descriptions:
                X_phrases_descriptions.append(each)
#                 if each[1] not in liste_entities_already_extracted:
                liste_entities_already_extracted.append(each[1])

CPU times: user 1min 6s, sys: 40 ms, total: 1min 6s
Wall time: 1min 6s


In [40]:
%%time
k = X_phrases_descriptions
import itertools
k.sort()
nodup_X_phrases_descriptions = list(k for k,_ in itertools.groupby(k))

CPU times: user 741 ms, sys: 2 µs, total: 741 ms
Wall time: 738 ms


In [41]:
%%time
k = liste_entities_already_extracted
import itertools
k.sort()
nodup_entities_already_extracted = list(k for k,_ in itertools.groupby(k))

CPU times: user 263 ms, sys: 0 ns, total: 263 ms
Wall time: 261 ms


In [33]:
# %%time
# for i, each in enumerate(nodup_X_phrases_descriptions,1):
#     if each != []:
#         print (each)
    
# ['ADJP', '%,bière blonde']
# ['ADJP', '&']
# ['ADJP', '& légumineuses']
# ['ADJP', ',']
# ['ADJP', ', active']
# ['ADJP', ', apéritives et toniques']
# ['ADJP', ', artisanale']
# ['ADJP', ', bienfaisantes']
# ['ADJP', ', biologique']
# ['ADJP', ', biologiques']
# ['ADJP', ', blanc']
# ['ADJP', ', blanche']
# ['ADJP', ', brut']
# ['ADJP', ', calmante et apaisante']
# ...
# ['ADJP', 'Assez riche']
# ['ADJP', 'Astucieux']
# ['ADJP', 'Attentive']
# ['ADJP', 'Aussi sain']
# ['ADJP', 'Authentique']
# ['ADJP', 'Authentique , végétale']
# ['ADJP', 'Açai,énergétique et tonifiante']
# ['ADJP', 'Beaucoup plus riche']
# ['ADJP', 'Blanc']
# ['ADJP', 'Bleu']
# ['ADJP', 'Bon']
# ['ADJP', 'Calme']
# ['ADJP', 'Charnue et fruitée']
# ['ADJP', 'Compatible']
# ['ADJP', 'Complet']
# ...
# ['NP', '2 pots Haricots Verts']
# ['NP', '2 pots Légumes Pâtes Jambon']
# ['NP', '2 pots Légumes Pâtes Poulet Bio']
# ['NP', '2 pots Petites pennes/Tomates/Courgettes']
# ['NP', '2 pots Petites pâtes']
# ['NP', '2 pots Pommes']
# ['NP', '2 pots Risotto']
# ['NP', '2 pots Tagliatelles/Epinards/Fromage']
# ['NP', '2 pots Tomates Pâtes Veau Bio']
# ...
# ['NP', 'un miel doux']
# ['NP', 'un miel typé']
# ['NP', 'un milieu']
# ['NP', 'un minimum']
# ['NP', 'un minéral']
# ['NP', 'un miso blanc très jeune']
# ['NP', 'un miso facile']
# ['NP', 'un mix']
# ['NP', 'un mix original']
# ['NP', 'un mode']
# ['NP', 'un moelleux']
# ...
# ['VP', 'vient compléter']
# ['VP', 'vient enrichir']
# ['VP', 'vient pas']
# ['VP', 'vient remplacer']
# ['VP', 'vient renforcer']
# ['VP', 'vient stimuler']
# ['VP', 'vignerons']
# ['VP', 'violacé']
# ['VP', 'violacée']
# ['VP', 'violacées']
# ['VP', 'violacés']

Ci-dessus: illustre bien que si on enlève les stopwords :<br>
On trouve en sortie les entités :<br>
    .......<br>
    .......<br>
    ...<br>

In [42]:
# %%time
# for i, each in enumerate(nodup_X_phrases_descriptions,1):
#     if each != []:
#         # Un type d'apostrophe
#         if "&rsquo" in each[0][1]:
# #         # Autre type d'apostrophe = quote qui est présente '
# #         if "'" in each[0][1]:
#             print (each)

CPU times: user 64 ms, sys: 0 ns, total: 64 ms
Wall time: 65.6 ms


In [ ]:
# Pas de trace du caractère spécial "&RSQUO" ou "&rsquo"

In [35]:
# %%time
# for i, each in enumerate(nodup_X_phrases_descriptions,1):
#     if each != []:
#         # Un type d'apostrophe
#         if "’" in each[0][1]:
# #         # Autre type d'apostrophe = quote qui est présente '
# #         if "'" in each[0][1]:
#             print (each)

CPU times: user 80 ms, sys: 0 ns, total: 80 ms
Wall time: 75.7 ms


In [ ]:
# df_descriptions_all_retailers[df_descriptions_all_retailers['description'].str.contains("’")]
# df_descriptions_all_retailers[df_descriptions_all_retailers['description'].str.contains("'")]

In [ ]:
# %%time
# # verifier que beacoup de ces cas disparaissent !!
# # OK plus aucun caractère spécial de ce type 
# for i, each in enumerate(nodup_X_phrases_descriptions,1):
#     if each != []:
#         if "&RSQUO" in each[0][1]:
#             print (each[0])

On récupère toutes les entités dans une liste, sans leurs TAGS

In [ ]:
# cf 
#  ['NP', 'Ce Château Cos'],
#  ['PP', "d'"],
#  ['NP', 'Estournel'],
    
# Il faudrait supprimer les PP ?
# Ou plutot dire que : NP + PP + NP = NP !
# en transformant la liste obtenue après application de la fonction "X_phrases_extraction"

In [42]:
%%time

liste_all = []
for each in nodup_X_phrases_descriptions:  
    liste_all.append(each[1])

CPU times: user 24.9 ms, sys: 0 ns, total: 24.9 ms
Wall time: 24.6 ms


#### NB) J'enlève "d'" du fichier des stopwords !
On avait avant modifification dans le fichier :<br>
187 completement<br>
188 d'<br>
189 d'abord<br>

In [ ]:
# import random
# test = random.sample(liste_all, 50)

In [43]:
%%time
##### Avant d'appliquer : nettoyer la liste dedup_X_phrases_titles avec cleansin_titles + remove stopwords
dedup_X_phrases_descriptions_to_use = []
for each in liste_all:
    dedup_X_phrases_descriptions_to_use.append(fd.remove_stopwords(fd.cleansing_titles(each)))

CPU times: user 2min 43s, sys: 2.82 s, total: 2min 46s
Wall time: 2min 46s


In [45]:
# %%time
# for i, each in enumerate(dedup_X_phrases_descriptions_to_use,1):
#     if each != []:
# #         if "&RSQUO" in each[0][1]:
#         print (each)
    
# , linoleique alpha-linolenique
# , leger , recyclable
# , legumineuses
# , legere
# , legere , ronde
# , legere rafraichissante
# , macrobiotiques
# , mental
# ...
# 100g.Tonique
# 110gTartinable
# 125gMendiants
# 130gPointes
# 150gMendiants
# 160gOlives vertes
# 190gJardiniere
# 190gOlives vertes entieres biologique
# 230gPratique
# 250gCereales
# 25clBiere
# ...
# l' association cereale
# l' association cereales
# l' association nutritionnelle cereale
# l' assurance
# l' astuce
# l' astuce ideale
# l' atelier
# l' atelier Vitamont
# l' atlantique
# l' atlantique Gros Danival
# ...
# pommes bio TROPICANA
# pommes bio concentrees.Poir'et pomme
# pommes bio francaise
# pommes bio100% fruit
# pommes bioGaz carbonique
# pommes biologiques
# pommes entieres
# pommes entieres issues
# pommes entieres.Il
# pommes framboises
# ...
# potager & Dinde 12M 230G Bio.Pour
# potager 1L Bio.L
# potager 50Cl Bio.Les chefs cuisiniers
# potager Dinde Romarinest
# potages
# potages , galette
# potages , galettes
# potages , gratins
# potages , pates vegetaux...).LesBioBisa
# potages , salades
# potages Hildegarde
# potages sauces.On
# ...
# poudre 125 g
# poudre 125G Bio.Moringa
# poudre 15G.D&RSQUO
# poudre 1er age BABYBIO
# poudre 1er age Bio CELIA LABORATOIRES
# poudre 1er age bio GUIGOZ
# poudre 1eme age HIPP BIOLOGIQUE
# poudre 2eme age bio HIPP BIOLOGIQUE
# poudre 2eme age BABYBIO
# poudre 2eme age Bio CELIA LABORATOIRES
# poudre 2eme age HIPP BIOLOGIQUE
# poudre 2eme age bio GUIGOZ

In [ ]:
# for each in dedup_X_phrases_descriptions_to_use:
#     if "&" in each:
#         print (each)

In [ ]:
# Regarder si "chateau cos d'estournel" est présent dans df_all_3_grams ?
# => alors NP + PP + NP est une entité ?

In [46]:
%%time

import numpy as np
array_X_phrases_descriptions = np.array(dedup_X_phrases_descriptions_to_use,dtype=list)

liste_X_phrases_found_in_1_grams_descriptions = []

for row in df_all_1_grams.itertuples():
    
    for X_phrases in array_X_phrases_descriptions :
        if X_phrases == row.un_grams and X_phrases not in liste_X_phrases_found_in_1_grams_descriptions:
            liste_X_phrases_found_in_1_grams_descriptions.append(X_phrases)

CPU times: user 28.2 s, sys: 3.98 ms, total: 28.2 s
Wall time: 28.2 s


In [47]:
%%time

import numpy as np
array_X_phrases_descriptions = np.array(dedup_X_phrases_descriptions_to_use,dtype=list)

liste_X_phrases_found_in_2_grams_descriptions = []

for row in df_all_2_grams.itertuples():
    
    for X_phrases in array_X_phrases_descriptions :
        if X_phrases == row.deux_grams and X_phrases not in liste_X_phrases_found_in_2_grams_descriptions:
            liste_X_phrases_found_in_2_grams_descriptions.append(X_phrases)

CPU times: user 1min 5s, sys: 7.88 ms, total: 1min 5s
Wall time: 1min 5s


In [68]:
# for each in liste_X_phrases_found_in_2_grams_descriptions:
#     if "l'" in each:
#         print (each)

In [48]:
%%time
liste_X_phrases_found_in_3_grams_descriptions = []

for row in df_all_3_grams.itertuples():   
    for X_phrases in array_X_phrases_descriptions:
        if  X_phrases == row.trois_grams  and X_phrases not in liste_X_phrases_found_in_3_grams_descriptions:
            liste_X_phrases_found_in_3_grams_descriptions.append(X_phrases)

CPU times: user 38 s, sys: 16 ms, total: 38 s
Wall time: 38 s


In [49]:
%%time
liste_X_phrases_found_in_4_grams_descriptions = []

for row in df_all_4_grams.itertuples():   
    for X_phrases in array_X_phrases_descriptions:
        if X_phrases == row.quatre_grams  and X_phrases not in liste_X_phrases_found_in_4_grams_descriptions:
            liste_X_phrases_found_in_4_grams_descriptions.append(X_phrases)

CPU times: user 17.4 s, sys: 0 ns, total: 17.4 s
Wall time: 17.4 s


#### step_3_find_xphrases_in_titles
Ici on trouve les entités présentes dans les titres

In [50]:
%%time
import numpy as np
array_X_phrases_1_grams_descriptions = np.array(liste_X_phrases_found_in_1_grams_descriptions,dtype=list)

import re
dict_result_1_grams = dict()

for i, row in enumerate(title_et_ngrams.itertuples(),1):
    liste_X_phrases_found = []
    X_phrases_found = dict()
    
    for X_phrases in array_X_phrases_1_grams_descriptions:
        if (np.char.rfind(np.array(row.title_without_stopwords_new,dtype=str),X_phrases) != -1 and re.search("(^|\s)" + re.escape(X_phrases) + "($|\s)",row.title_without_stopwords_new)):
           
            if X_phrases not in liste_X_phrases_found:
                liste_X_phrases_found.append(X_phrases)
                X_phrases_found[(row.slug, row.distributeur, row.title_without_stopwords_new)] = liste_X_phrases_found
                 
    if i == 1:
        dict_result_1_grams = X_phrases_found
    else:
        dict_result_1_grams = {**dict_result_1_grams, **X_phrases_found}
        
del liste_X_phrases_found, X_phrases_found, X_phrases
df_entites_X_phrases_1_grams_found = pd.DataFrame(list(dict_result_1_grams.items()), columns = ['id','entity_X_phrases_1_grams_found'])

CPU times: user 2min 47s, sys: 124 ms, total: 2min 47s
Wall time: 2min 47s


In [51]:
%%time
import numpy as np
array_X_phrases_2_grams_descriptions = np.array(liste_X_phrases_found_in_2_grams_descriptions,dtype=list)

import re
dict_result_2_grams = dict()

for i, row in enumerate(title_et_ngrams.itertuples(),1):
    liste_X_phrases_found = []
    X_phrases_found = dict()
    
    for X_phrases in array_X_phrases_2_grams_descriptions:
        if (np.char.rfind(np.array(row.title_without_stopwords_new,dtype=str),X_phrases) != -1 and re.search("(^|\s)" + re.escape(X_phrases) + "($|\s)",row.title_without_stopwords_new)):
           
            if X_phrases not in liste_X_phrases_found:
                liste_X_phrases_found.append(X_phrases)
                X_phrases_found[(row.slug, row.distributeur, row.title_without_stopwords_new)] = liste_X_phrases_found
                 
    if i == 1:
        dict_result_2_grams = X_phrases_found
    else:
        dict_result_2_grams = {**dict_result_2_grams, **X_phrases_found}
        
del liste_X_phrases_found, X_phrases_found, X_phrases
df_entites_X_phrases_2_grams_found = pd.DataFrame(list(dict_result_2_grams.items()), columns = ['id','entity_X_phrases_2_grams_found'])

CPU times: user 1min 2s, sys: 39.9 ms, total: 1min 2s
Wall time: 1min 2s


In [74]:
# Ou est Château Cos d'Estournel => n'est pas dans "liste_X_phrases_found_in_3_grams_descriptions"
# sorted(liste_X_phrases_found_in_3_grams_descriptions)

In [53]:
%%time
import numpy as np
array_X_phrases_3_grams_descriptions = np.array(liste_X_phrases_found_in_3_grams_descriptions,dtype=list)

import re
dict_result_3_grams = dict()

for i, row in enumerate(title_et_ngrams.itertuples(),1):
    
    liste_X_phrases_found = []
    X_phrases_found = dict()

    for X_phrases in array_X_phrases_3_grams_descriptions:
        if (np.char.rfind(np.array(row.title_without_stopwords_new,dtype=str),X_phrases) != -1 and re.search("(^|\s)" + re.escape(X_phrases) + "($|\s)",row.title_without_stopwords_new)):
            
            if X_phrases not in liste_X_phrases_found:
                liste_X_phrases_found.append(X_phrases)
                X_phrases_found[(row.slug, row.distributeur, row.title_without_stopwords_new)] = liste_X_phrases_found
                 
    if i == 1:
        dict_result_3_grams = X_phrases_found
    else:
        dict_result_3_grams = {**dict_result_3_grams, **X_phrases_found}
        
del liste_X_phrases_found, X_phrases_found, X_phrases 

df_entites_X_phrases_3_grams_found = pd.DataFrame(list(dict_result_3_grams.items()), columns = ['id','entity_X_phrases_3_grams_found'])

CPU times: user 8.66 s, sys: 0 ns, total: 8.66 s
Wall time: 8.66 s


In [77]:
# Ou est Château Cos d'Estournel
# df_entites_X_phrases_3_grams_found.sample(n=5)
# df_entites_X_phrases_3_grams_found[df_entites_X_phrases_3_grams_found['slug'].str.contains('cos')]

In [54]:
%%time
# import numpy as np
array_X_phrases_4_grams_descriptions = np.array(liste_X_phrases_found_in_4_grams_descriptions,dtype=list)

# import re
dict_result_4_grams = {}
for i, row in enumerate(title_et_ngrams.itertuples(),1):

    liste_X_phrases_found = []
    X_phrases_found = {}
    for X_phrases in array_X_phrases_4_grams_descriptions:
 
        if (np.char.rfind(np.array(row.title_without_stopwords_new,dtype=str),X_phrases) != -1 and re.search("(^|\s)" + re.escape(X_phrases) + "($|\s)",row.title_without_stopwords_new)):
            if X_phrases not in liste_X_phrases_found:
                liste_X_phrases_found.append(X_phrases)
                X_phrases_found[(row.slug, row.distributeur, row.title_without_stopwords_new)] = liste_X_phrases_found
                 
    if i == 1:
        dict_result_4_grams = X_phrases_found
    else:
        dict_result_4_grams = {**dict_result_4_grams, **X_phrases_found}
        
del liste_X_phrases_found, X_phrases_found, X_phrases 

df_entites_X_phrases_4_grams_found = pd.DataFrame(list(dict_result_4_grams.items()), columns = ['id','entity_X_phrases_4_grams_found'])

CPU times: user 1.02 s, sys: 0 ns, total: 1.02 s
Wall time: 1.02 s


#### step_4_join_entities_extracted

In [55]:
df_entites_X_phrases_1_grams_found['slug'] = df_entites_X_phrases_1_grams_found['id'].map(lambda x: x[0])

In [56]:
df_entites_X_phrases_2_grams_found['slug'] = df_entites_X_phrases_2_grams_found['id'].map(lambda x: x[0])
df_entites_X_phrases_3_grams_found['slug'] = df_entites_X_phrases_3_grams_found['id'].map(lambda x: x[0])
df_entites_X_phrases_4_grams_found['slug'] = df_entites_X_phrases_4_grams_found['id'].map(lambda x: x[0])

In [57]:
merged_results = pd.merge(df_entites_X_phrases_2_grams_found, df_entites_X_phrases_3_grams_found[['slug','entity_X_phrases_3_grams_found']], on = 'slug', how = 'left')
# merged_results = pd.merge(merged_results, df_entites_X_phrases_3_grams_found[['slug','entity_X_phrases_3_grams_found']], on = 'slug', how = 'left')
merged_results = pd.merge(merged_results, df_entites_X_phrases_4_grams_found[['slug','entity_X_phrases_4_grams_found']], on = 'slug', how = 'left')
# all_results = merged_results[['id','slug','entity_X_phrases_1_grams_found','entity_X_phrases_2_grams_found','entity_X_phrases_3_grams_found','entity_X_phrases_4_grams_found']]
all_results = merged_results[['id','slug','entity_X_phrases_2_grams_found','entity_X_phrases_3_grams_found','entity_X_phrases_4_grams_found']]

In [58]:
all_results = all_results.fillna("NA")

In [59]:
all_results.shape

(7606, 5)

In [60]:
# A faire au début, pour utiliser "%autoreload" : 
%load_ext autoreload
%autoreload

import pandas as pd
pd.set_option('display.max_colwidth',-1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
# list(all_results)

['id',
 'slug',
 'entity_X_phrases_2_grams_found',
 'entity_X_phrases_3_grams_found',
 'entity_X_phrases_4_grams_found']

#### step_5_ngrams_overlapped_and_not_overlapped

In [62]:
from functions import fun_step_5_ngrams_overlapped_and_not_overlapped as fng

# On crée une colonne avec toutes les "entity_X trouvées"
all_results['all_entity_X_phrases']= all_results.apply(fng.all_ngrams_X_phrases, axis = 1)

In [63]:
# all_results.shape

(7606, 6)

In [64]:
all_results

,id,slug,entity_X_phrases_2_grams_found,entity_X_phrases_3_grams_found,entity_X_phrases_4_grams_found,all_entity_X_phrases
0,"(fromage-bio-frais-carrefour-bio-3245413442420, crf, fromage bio frais carrefour bio)",fromage-bio-frais-carrefour-bio-3245413442420,[fromage bio],NA,NA,[fromage bio]
1,"(cafe-bio-equilibre-carrefour-bio-3560070712786, crf, cafe bio equilibre carrefour bio)",cafe-bio-equilibre-carrefour-bio-3560070712786,[cafe bio],NA,NA,[cafe bio]
2,"(pain-bio-cereales-carrefour-bio-3523680347823, crf, pain bio cereales carrefour bio)",pain-bio-cereales-carrefour-bio-3523680347823,"[pain bio, bio cereales]",NA,NA,"[pain bio, bio cereales]"
3,"(fromage-bio-frais-carrefour-bio-3560070542611, crf, fromage bio frais carrefour bio)",fromage-bio-frais-carrefour-bio-3560070542611,[fromage bio],NA,NA,[fromage bio]
4,"(fromage-bio-feta-carrefour-bio-3560070530595, crf, fromage bio feta carrefour bio)",fromage-bio-feta-carrefour-bio-3560070530595,[fromage bio],NA,NA,[fromage bio]
5,"(pates-bio-spaghetti-carrefour-bio-3270190127833, crf, pates bio spaghetti carrefour bio)",pates-bio-spaghetti-carrefour-bio-3270190127833,[pates bio],NA,NA,[pates bio]
6,"(sauce-bio-mayonnaise-carrefour-bio-3560070503445, crf, sauce bio mayonnaise carrefour bio)",sauce-bio-mayonnaise-carrefour-bio-3560070503445,[sauce bio],NA,NA,[sauce bio]
7,"(haricot-vert-bio-3276552687113, crf, haricot vert bio)",haricot-vert-bio-3276552687113,"[vert bio, haricot vert]",NA,NA,"[vert bio, haricot vert]"
8,"(haricot-vert-bio-3270190211488, crf, haricot vert bio)",haricot-vert-bio-3270190211488,"[vert bio, haricot vert]",NA,NA,"[vert bio, haricot vert]"
9,"(haricots-verts-bio-tres-fins-3760013350536, crf, haricots verts bio fins)",haricots-verts-bio-tres-fins-3760013350536,[haricots verts],[haricots verts bio],NA,"[haricots verts, haricots verts bio]"


In [68]:
# 10/09/2018
# 1. combinaison des plus ngrams qui ne s'overlappent pas
#  1.1 possibilité de favoriser les ngrams les plus longs
#  1.2 possibilité de favoriser les ngrams plus cours
# 2. si que des inclusions, on prend le plus long ngrams

# 3. Choix des combinaisons avec somme des fréquences maximum = vraiment appliqué ?

In [88]:
# all_results.sample(n=10)[['all_entity_X_phrases']]

In [65]:
all_results['not_overlapped_entity_X_phrases']= all_results['all_entity_X_phrases'].map(fng.not_overlap_ngrams)

In [67]:
# import nltk
# nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hapax94/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [66]:
all_results['before_overlaped']= all_results.apply(fng.remove_ngrams_with_no_overlap, axis = 1)

In [68]:
all_results['overlapped_entity_X_phrases']= all_results['before_overlaped'].map(fng.overlap_ngrams)

In [70]:
# all_results.sample(n=1)

,id,slug,entity_X_phrases_2_grams_found,entity_X_phrases_3_grams_found,entity_X_phrases_4_grams_found,all_entity_X_phrases,not_overlapped_entity_X_phrases,before_overlaped,overlapped_entity_X_phrases
358,"(galettes-mais-chocolat-au-lait-bio-bjorg-3229820769141, crf, galettes chocolat lait bio bjorg)",galettes-mais-chocolat-au-lait-bio-bjorg-3229820769141,"[chocolat lait, lait bio]",NA,NA,"[chocolat lait, lait bio]",[],"[chocolat lait, lait bio]","[chocolat lait, lait bio]"


In [71]:
# On prend le ngrams qui contient tous les autres à partir de "overlapped_entity_X_phrases"
# pour chaque ngrams de la la liste des ngrams :
#     si un ngrams donné contient tous les autres ngrams alors je le garde !!!           
all_results['keep_overlapped_entity_X_phrases']= all_results['overlapped_entity_X_phrases'].map(fng.keep_ngrams_which_contains_others)                                           

#### step_6_get_results

In [72]:
from functions import fun_step_6_get_results as fg

all_results['final_result_X_phrases'] = all_results.apply(fg.final_result, axis = 1)

In [73]:
# all_results.sample(n=1)

,id,slug,entity_X_phrases_2_grams_found,entity_X_phrases_3_grams_found,entity_X_phrases_4_grams_found,all_entity_X_phrases,not_overlapped_entity_X_phrases,before_overlaped,overlapped_entity_X_phrases,keep_overlapped_entity_X_phrases,final_result_X_phrases
1464,"(plat-cuisine-tofu-a-l-indienne-carrefour-bio-3560070575855, crf, plat cuisine tofu a l'indienne carrefour bio)",plat-cuisine-tofu-a-l-indienne-carrefour-bio-3560070575855,[plat cuisine],NA,NA,[plat cuisine],[plat cuisine],[],[],[],[plat cuisine]


In [74]:
# Ajout de la colonne "title" sans l'année
import re

all_results["title_temp"] = all_results['id'].map(lambda x: x[2] if x != 'NA' else x)
all_results["title"] = all_results['title_temp'].map(lambda x: re.sub('\d{4}','',x))
all_results = all_results.drop(['title_temp'], axis =1)

In [75]:
# Calcul de la différence "title" - "final_result_X_phrases" :
all_results["difference_title_vs_final_result_X_phrases"] = all_results.apply(fg.difference_title_vs_final_result_X_phrases, axis = 1)

In [76]:
# Ajout de l'entité 1_grams si on en a trouvé une 
all_results["final_result_X_phrases_bis"] = all_results.apply(fg.add_entity_1_grams, axis = 1)

In [9]:
# import pickle
# import pandas as pd
# data_folder = '/home/hapax94/Documents/vincent/jupyter/carrefour_1/notebooks/github_part2/analyse_keywords_with_descriptions/pattern/data'
# all_results.to_pickle(data_folder + '/all_results_151118.pkl')
# all_results =  pd.read_pickle(data_folder + "/all_results_151118.pkl")

#### step_7_comparing_with_truth_from_properties

In [ ]:
Je lance le dernier code cette fois avec pour périmètre tout le dataset des résultats<br>
Soit près de 7600 produits BIO :

In [79]:
all_results

,id,slug,entity_X_phrases_2_grams_found,entity_X_phrases_3_grams_found,entity_X_phrases_4_grams_found,all_entity_X_phrases,not_overlapped_entity_X_phrases,before_overlaped,overlapped_entity_X_phrases,keep_overlapped_entity_X_phrases,final_result_X_phrases,title,difference_title_vs_final_result_X_phrases,final_result_X_phrases_bis
0,"(fromage-bio-frais-carrefour-bio-3245413442420, crf, fromage bio frais carrefour bio)",fromage-bio-frais-carrefour-bio-3245413442420,[fromage bio],NA,NA,[fromage bio],[fromage bio],[],[],[],[fromage bio],fromage bio frais carrefour bio,frais carrefour bio,[fromage bio]
1,"(cafe-bio-equilibre-carrefour-bio-3560070712786, crf, cafe bio equilibre carrefour bio)",cafe-bio-equilibre-carrefour-bio-3560070712786,[cafe bio],NA,NA,[cafe bio],[cafe bio],[],[],[],[cafe bio],cafe bio equilibre carrefour bio,equilibre carrefour bio,[cafe bio]
2,"(pain-bio-cereales-carrefour-bio-3523680347823, crf, pain bio cereales carrefour bio)",pain-bio-cereales-carrefour-bio-3523680347823,"[pain bio, bio cereales]",NA,NA,"[pain bio, bio cereales]",[],"[pain bio, bio cereales]","[pain bio, bio cereales]",[],[],pain bio cereales carrefour bio,pain bio cereales carrefour bio,[]
3,"(fromage-bio-frais-carrefour-bio-3560070542611, crf, fromage bio frais carrefour bio)",fromage-bio-frais-carrefour-bio-3560070542611,[fromage bio],NA,NA,[fromage bio],[fromage bio],[],[],[],[fromage bio],fromage bio frais carrefour bio,frais carrefour bio,[fromage bio]
4,"(fromage-bio-feta-carrefour-bio-3560070530595, crf, fromage bio feta carrefour bio)",fromage-bio-feta-carrefour-bio-3560070530595,[fromage bio],NA,NA,[fromage bio],[fromage bio],[],[],[],[fromage bio],fromage bio feta carrefour bio,feta carrefour bio,[fromage bio]
5,"(pates-bio-spaghetti-carrefour-bio-3270190127833, crf, pates bio spaghetti carrefour bio)",pates-bio-spaghetti-carrefour-bio-3270190127833,[pates bio],NA,NA,[pates bio],[pates bio],[],[],[],[pates bio],pates bio spaghetti carrefour bio,spaghetti carrefour bio,[pates bio]
6,"(sauce-bio-mayonnaise-carrefour-bio-3560070503445, crf, sauce bio mayonnaise carrefour bio)",sauce-bio-mayonnaise-carrefour-bio-3560070503445,[sauce bio],NA,NA,[sauce bio],[sauce bio],[],[],[],[sauce bio],sauce bio mayonnaise carrefour bio,mayonnaise carrefour bio,[sauce bio]
7,"(haricot-vert-bio-3276552687113, crf, haricot vert bio)",haricot-vert-bio-3276552687113,"[vert bio, haricot vert]",NA,NA,"[vert bio, haricot vert]",[],"[vert bio, haricot vert]","[vert bio, haricot vert]",[],[],haricot vert bio,haricot vert bio,[]
8,"(haricot-vert-bio-3270190211488, crf, haricot vert bio)",haricot-vert-bio-3270190211488,"[vert bio, haricot vert]",NA,NA,"[vert bio, haricot vert]",[],"[vert bio, haricot vert]","[vert bio, haricot vert]",[],[],haricot vert bio,haricot vert bio,[]
9,"(haricots-verts-bio-tres-fins-3760013350536, crf, haricots verts bio fins)",haricots-verts-bio-tres-fins-3760013350536,[haricots verts],[haricots verts bio],NA,"[haricots verts, haricots verts bio]",[],"[haricots verts, haricots verts bio]","[haricots verts, haricots verts bio]",[haricots verts bio],[haricots verts bio],haricots verts bio fins,fins,"[haricots verts bio, fins]"


SAUVEGARDE

In [268]:
# del all_results

In [80]:
# data_folder = "/home/hapax94/Documents/vincent/jupyter/carrefour_bio/output"

# all_results.to_pickle(data_folder + '/all_results_BIO_310119_17h.pkl')
# all_results = pd.read_pickle(data_folder + '/all_results_BIO_310119.pkl')
# all_results

In [81]:
import re
# On regarde le % de titres, parmi 9082 vins, dont le découpage en entités permet de mapper complètement tous les mots des titres dans les millésimes
df_temp_bis = all_results[['id','slug','final_result_X_phrases_bis']]

In [82]:
df_temp_bis['title_without_year'] = all_results['id'].map(lambda x: x[2]).map(lambda x: re.sub('\d{4}','',x))
df_temp_bis = df_temp_bis.fillna("NA")

/home/hapax94/.pyenv/versions/my_env_new/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [ ]:
ATTENTION, on a besoin des fonctions :
- flag_result_decoupage_entier
- list_entity_to_str
- nodup_liste
- extract_wines_final

In [83]:
def flag_result_decoupage_entier(df):
    
    nb_words_final_result = 0
    for each in df['final_result_X_phrases_bis']:
        words_each = each.split(' ')
        nb_words_final_result += len(words_each)
#     print ("Pour", df['final_result_X_phrases_bis'],"on a", nb_words_final_result)
     
    nb_words_title = 0
    words_title = df['title_without_year'].split(' ')
    for each in words_title:
        if re.sub('\s+',' ',each):
#             print (each)
            nb_words_title += 1      
#     print ("Pour", df['title_without_year'],"on a", nb_words_title)
        
    if nb_words_final_result == nb_words_title: 
        resultat = 'Yes'
    else:
        resultat = 'No'

    return resultat

In [84]:
def list_entity_to_str(x):
    i = 0
    for each in x:
        i += 1
        if i == 1:
            resultat = each
        else:
            resultat = resultat + '|' + each
    return resultat

In [85]:
def nodup_liste(x):
    resultat = []
    for each in x:
        if each not in resultat:
            resultat.append(each)
    
    return resultat

In [86]:
df_temp_bis['flag_decoupage_entier'] = df_temp_bis.apply(flag_result_decoupage_entier, axis=1)
(len(df_temp_bis[df_temp_bis['flag_decoupage_entier'] == 'Yes'])/len(df_temp_bis))*100
len(df_temp_bis)
df_with_100_pct_bis = df_temp_bis[df_temp_bis['flag_decoupage_entier'] == 'Yes']

In [87]:
df_with_100_pct_bis['str_result'] = df_with_100_pct_bis['final_result_X_phrases_bis'].map(list_entity_to_str)
df_with_100_pct_bis['liste_id'] = df_with_100_pct_bis['id'].apply(list)
df_get_matching_bio_bis = df_with_100_pct_bis.groupby(['str_result'])['liste_id'].apply(list).to_frame()

/home/hapax94/.pyenv/versions/my_env_new/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/hapax94/.pyenv/versions/my_env_new/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


31/01/2019 : la fonction "extract_wines_final" est à modifier pour le BIO 

In [88]:
df_get_matching_bio_bis['len_id'] = df_get_matching_bio_bis['liste_id'].map(len)
df_get_matching_bio_bis = df_get_matching_bio_bis[df_get_matching_bio_bis['len_id'] > 1]
df_get_matching_bio_bis['nodup_liste_id'] = df_get_matching_bio_bis['liste_id'].map(nodup_liste)

In [89]:
df_get_matching_bio_bis

,liste_id,len_id,nodup_liste_id
str_result,,,
1 kg|ble bio|farine|t65,"[[le-moulin-du-pere-martin-farine-de-ble-bio-t65-1-kg-p96985, gwz, farine ble bio t65 1 kg], [mon-fournil-farine-de-ble-bio-t65-1-kg-p75109, gwz, farine ble bio t65 1 kg]]",2,"[[le-moulin-du-pere-martin-farine-de-ble-bio-t65-1-kg-p96985, gwz, farine ble bio t65 1 kg], [mon-fournil-farine-de-ble-bio-t65-1-kg-p75109, gwz, farine ble bio t65 1 kg]]"
10 mois|lait|bebe|liquide|babybio,"[[lait-bebe-liquide-des-10-mois-babybio-3288131500072, crf, lait bebe liquide 10 mois babybio], [lait-bebe-liquide-des-10-mois-babybio-3288131500065, crf, lait bebe liquide 10 mois babybio]]",2,"[[lait-bebe-liquide-des-10-mois-babybio-3288131500072, crf, lait bebe liquide 10 mois babybio], [lait-bebe-liquide-des-10-mois-babybio-3288131500065, crf, lait bebe liquide 10 mois babybio]]"
10 mois|lait|bebe|liquide|bio|candia,"[[lait-bebe-liquide-bio-des-10-mois-candia-3533631254001, crf, lait bebe liquide bio 10 mois candia], [lait-bebe-liquide-bio-des-10-mois-candia-3533631255008, crf, lait bebe liquide bio 10 mois candia]]",2,"[[lait-bebe-liquide-bio-des-10-mois-candia-3533631254001, crf, lait bebe liquide bio 10 mois candia], [lait-bebe-liquide-bio-des-10-mois-candia-3533631255008, crf, lait bebe liquide bio 10 mois candia]]"
12 mois|plat|bebe|bio|legumes/veau|good|gout,"[[plat-bebe-bio-des-12-mois-legumes-veau-good-gout-3770002327678, crf, plat bebe bio 12 mois legumes/veau good gout], [plat-bebe-bio-des-12-mois-legumes-veau-good-gout-3760269310049, crf, plat bebe bio 12 mois legumes/veau good gout]]",2,"[[plat-bebe-bio-des-12-mois-legumes-veau-good-gout-3770002327678, crf, plat bebe bio 12 mois legumes/veau good gout], [plat-bebe-bio-des-12-mois-legumes-veau-good-gout-3760269310049, crf, plat bebe bio 12 mois legumes/veau good gout]]"
1er age|lait|bebe|poudre|babybio,"[[lait-bebe-en-poudre-1er-age-babybio-3288131580012, crf, lait bebe poudre 1er age babybio], [lait-bebe-en-poudre-1er-age-babybio-3288131500041, crf, lait bebe poudre 1er age babybio]]",2,"[[lait-bebe-en-poudre-1er-age-babybio-3288131580012, crf, lait bebe poudre 1er age babybio], [lait-bebe-en-poudre-1er-age-babybio-3288131500041, crf, lait bebe poudre 1er age babybio]]"
20 sachets|camomille bio|infusion,"[[bio-conseils-infusion-camomille-bio-20-sachets-p87772, gwz, infusion camomille bio 20 sachets], [nutrisensis-infusion-camomille-bio-20-sachets-p81457, gwz, infusion camomille bio 20 sachets]]",2,"[[bio-conseils-infusion-camomille-bio-20-sachets-p87772, gwz, infusion camomille bio 20 sachets], [nutrisensis-infusion-camomille-bio-20-sachets-p81457, gwz, infusion camomille bio 20 sachets]]"
20 sachets|romarin bio|infusion,"[[bio-conseils-infusion-de-romarin-bio-20-sachets-p87822, gwz, infusion romarin bio 20 sachets], [nutrisensis-infusion-romarin-bio-20-sachets-p81468, gwz, infusion romarin bio 20 sachets]]",2,"[[bio-conseils-infusion-de-romarin-bio-20-sachets-p87822, gwz, infusion romarin bio 20 sachets], [nutrisensis-infusion-romarin-bio-20-sachets-p81468, gwz, infusion romarin bio 20 sachets]]"
20 sachets|tilleul bio|infusion,"[[bio-conseils-infusion-de-tilleul-bio-20-sachets-p87850, gwz, infusion tilleul bio 20 sachets], [nutrisensis-infusion-tilleul-bio-20-sachets-p81472, gwz, infusion tilleul bio 20 sachets]]",2,"[[bio-conseils-infusion-de-tilleul-bio-20-sachets-p87850, gwz, infusion tilleul bio 20 sachets], [nutrisensis-infusion-tilleul-bio-20-sachets-p81472, gwz, infusion tilleul bio 20 sachets]]"
20 sachets|vert matcha|the|supreme|bio,"[[pukka-the-vert-matcha-supreme-bio-20-sachets-p100327, gwz, the vert matcha supreme bio 20 sachets], [the-vert-matcha-supreme-20inf, ntr, the vert matcha supreme 20 sachets bio]]",2,"[[pukka-the-vert-matcha-supreme-bio-20-sachets-p100327, gwz, the vert matcha supreme bio 20 sachets], [the-vert-matcha-supreme-20inf, ntr, the vert matcha supreme 20 sachets bio]]"


In [90]:
pattern_year = re.compile('\d{4}')
def extract_bio_final(x):
    
    resultat = []

    for i in range(len(x)):
        for j in range(len(x)):
            if (i != j):
#                 annee_i = pattern_year.findall(x[i][2])
#                 annee_j = pattern_year.findall(x[j][2])

#                 if annee_i == []:
#                     annee_i = pattern_year.findall(x[i][0])
                      
#                 if annee_j == []:
#                     annee_j = pattern_year.findall(x[j][0])
                
#                 if  x[i][0] == "chateau-la-dominique-2011" or x[j][0] == "chateau-la-dominique-2011":
#                     print ("i", x[i])
#                     print ("j",x[j])
#                 if x[i][1] != x[j][1] and annee_i == annee_j:
                if x[i][1] != x[j][1]:
    #                   print ("Distributeurs différents et memes années !")
    #                   print ((x[i],x[j]))
                    if x[i] not in resultat:
                        resultat.append(x[i])
                    if x[j] not in resultat:
                        resultat.append(x[j])
                
    return resultat

In [91]:
df_get_matching_bio_bis['RESULTAT'] = df_get_matching_bio_bis['nodup_liste_id'].map(extract_bio_final)

df_get_matching_bio_bis['len_resultat'] = df_get_matching_bio_bis['RESULTAT'].map(lambda x: len(x))
# df_get_matching_bio_bis[df_get_matching_bio_bis['len_resultat'] != 0]

In [92]:
df_get_matching_bio_bis

,liste_id,len_id,nodup_liste_id,RESULTAT,len_resultat
str_result,,,,,
1 kg|ble bio|farine|t65,"[[le-moulin-du-pere-martin-farine-de-ble-bio-t65-1-kg-p96985, gwz, farine ble bio t65 1 kg], [mon-fournil-farine-de-ble-bio-t65-1-kg-p75109, gwz, farine ble bio t65 1 kg]]",2,"[[le-moulin-du-pere-martin-farine-de-ble-bio-t65-1-kg-p96985, gwz, farine ble bio t65 1 kg], [mon-fournil-farine-de-ble-bio-t65-1-kg-p75109, gwz, farine ble bio t65 1 kg]]",[],0
10 mois|lait|bebe|liquide|babybio,"[[lait-bebe-liquide-des-10-mois-babybio-3288131500072, crf, lait bebe liquide 10 mois babybio], [lait-bebe-liquide-des-10-mois-babybio-3288131500065, crf, lait bebe liquide 10 mois babybio]]",2,"[[lait-bebe-liquide-des-10-mois-babybio-3288131500072, crf, lait bebe liquide 10 mois babybio], [lait-bebe-liquide-des-10-mois-babybio-3288131500065, crf, lait bebe liquide 10 mois babybio]]",[],0
10 mois|lait|bebe|liquide|bio|candia,"[[lait-bebe-liquide-bio-des-10-mois-candia-3533631254001, crf, lait bebe liquide bio 10 mois candia], [lait-bebe-liquide-bio-des-10-mois-candia-3533631255008, crf, lait bebe liquide bio 10 mois candia]]",2,"[[lait-bebe-liquide-bio-des-10-mois-candia-3533631254001, crf, lait bebe liquide bio 10 mois candia], [lait-bebe-liquide-bio-des-10-mois-candia-3533631255008, crf, lait bebe liquide bio 10 mois candia]]",[],0
12 mois|plat|bebe|bio|legumes/veau|good|gout,"[[plat-bebe-bio-des-12-mois-legumes-veau-good-gout-3770002327678, crf, plat bebe bio 12 mois legumes/veau good gout], [plat-bebe-bio-des-12-mois-legumes-veau-good-gout-3760269310049, crf, plat bebe bio 12 mois legumes/veau good gout]]",2,"[[plat-bebe-bio-des-12-mois-legumes-veau-good-gout-3770002327678, crf, plat bebe bio 12 mois legumes/veau good gout], [plat-bebe-bio-des-12-mois-legumes-veau-good-gout-3760269310049, crf, plat bebe bio 12 mois legumes/veau good gout]]",[],0
1er age|lait|bebe|poudre|babybio,"[[lait-bebe-en-poudre-1er-age-babybio-3288131580012, crf, lait bebe poudre 1er age babybio], [lait-bebe-en-poudre-1er-age-babybio-3288131500041, crf, lait bebe poudre 1er age babybio]]",2,"[[lait-bebe-en-poudre-1er-age-babybio-3288131580012, crf, lait bebe poudre 1er age babybio], [lait-bebe-en-poudre-1er-age-babybio-3288131500041, crf, lait bebe poudre 1er age babybio]]",[],0
20 sachets|camomille bio|infusion,"[[bio-conseils-infusion-camomille-bio-20-sachets-p87772, gwz, infusion camomille bio 20 sachets], [nutrisensis-infusion-camomille-bio-20-sachets-p81457, gwz, infusion camomille bio 20 sachets]]",2,"[[bio-conseils-infusion-camomille-bio-20-sachets-p87772, gwz, infusion camomille bio 20 sachets], [nutrisensis-infusion-camomille-bio-20-sachets-p81457, gwz, infusion camomille bio 20 sachets]]",[],0
20 sachets|romarin bio|infusion,"[[bio-conseils-infusion-de-romarin-bio-20-sachets-p87822, gwz, infusion romarin bio 20 sachets], [nutrisensis-infusion-romarin-bio-20-sachets-p81468, gwz, infusion romarin bio 20 sachets]]",2,"[[bio-conseils-infusion-de-romarin-bio-20-sachets-p87822, gwz, infusion romarin bio 20 sachets], [nutrisensis-infusion-romarin-bio-20-sachets-p81468, gwz, infusion romarin bio 20 sachets]]",[],0
20 sachets|tilleul bio|infusion,"[[bio-conseils-infusion-de-tilleul-bio-20-sachets-p87850, gwz, infusion tilleul bio 20 sachets], [nutrisensis-infusion-tilleul-bio-20-sachets-p81472, gwz, infusion tilleul bio 20 sachets]]",2,"[[bio-conseils-infusion-de-tilleul-bio-20-sachets-p87850, gwz, infusion tilleul bio 20 sachets], [nutrisensis-infusion-tilleul-bio-20-sachets-p81472, gwz, infusion tilleul bio 20 sachets]]",[],0
20 sachets|vert matcha|the|supreme|bio,"[[pukka-the-vert-matcha-supreme-bio-20-sachets-p100327, gwz, the vert matcha supreme bio 20 sachets], [the-vert-matcha-supreme-20inf, ntr, the vert matcha supreme 20 sachets bio]]",2,"[[pukka-the-vert-matcha-supreme-bio-20-sachets-p100327, gwz, the vert matcha supreme bio 20 sachets], [the-vert-matcha-supreme-20inf, ntr, the vert matcha supreme 20 sachets bio]]","[[pukka-the-vert-matcha-supreme-bio-20-sachets-p1

In [95]:
df_resultat = df_get_matching_bio_bis[df_get_matching_bio_bis['len_resultat'] != 0][['RESULTAT']]

In [96]:
df_resultat

,RESULTAT
str_result,
20 sachets|vert matcha|the|supreme|bio,"[[pukka-the-vert-matcha-supreme-bio-20-sachets-p100327, gwz, the vert matcha supreme bio 20 sachets], [the-vert-matcha-supreme-20inf, ntr, the vert matcha supreme 20 sachets bio]]"
chocolat lait|100g,"[[saveurs-nature-chocolat-au-lait-100g-p89890, gwz, chocolat lait 100g], [chocolat-lait-100g, ntr, chocolat lait 100g]]"
couscous demi-complet|500g,"[[markal-couscous-demi-complet-500g-p76290, gwz, couscous demi-complet 500g], [couscous-1-2-cpt-500g, ntr, couscous demi-complet 500g]]"
lentilles vertes|petits legumes|cuisinees,"[[prosain-lentilles-vertes-cuisinees-aux-petits-legumes-p69328, gwz, lentilles vertes cuisinees petits legumes], [8144-lentilles-vertes-bio-cuisinees-aux-petits-legumes-3335880004716?PageSpeed=noscript, wbe, lentilles vertes cuisinees petits legumes], [4083-lentilles-vertes-bio-cuisinees-aux-petits-legumes-3335880006031?PageSpeed=noscript, wbe, lentilles vertes cuisinees petits legumes]]"
sesame noir|250g,"[[markal-sesame-noir-250g-p77439, gwz, sesame noir 250g], [sesame-noir-250g, ntr, sesame noir 250g]]"


In [129]:
# pattern_year = re.compile('\d{4}')
# def add_year(x):
#     resultat = []
    
#     for each in x:
        
#         if pattern_year.findall(each[2]) != []:
#             # On crée une liste "each_new" qui est une copie indépendante de "each"
#             each_new = each[:]
#             each_new.append(pattern_year.findall(each[2])[0])
#         else:
#             if pattern_year.findall(each[0]) != []:
#                 # On crée une liste "each_new" qui est une copie indépendante de "each"
#                 each_new = each[:]
#                 each_new.append(pattern_year.findall(each[0])[0])
#             else:
#                 each_new = each[:]
#                 each_new.append('')
        
#         resultat.append(each_new)
        
#     return resultat

# df_resultat['RESULTAT_with_year_added'] = df_resultat['RESULTAT'].map(add_year)

In [97]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

In [98]:
dict_slug_to_url = dict(zip(orgc_descriptions.xslug,orgc_descriptions.xurl))
def urls(x):
    resultat = []
    for each in x:
        element = []
        element.append(dict_slug_to_url[each[0]])
        resultat.append(element)
    return resultat

In [99]:
# df_resultat['RESULTAT_sorted_by_year'] = df_resultat['RESULTAT_with_year_added'].map(sort_by_year)
# df_resultat['RESULTAT_FINAL'] = df_resultat['RESULTAT_sorted_by_year'].map(remove_info_year)
# df_resultat['URLS'] = df_resultat['RESULTAT_FINAL'].map(urls)
# df_resultat = df_resultat[['RESULTAT_FINAL','URLS']]
df_resultat['URLS'] = df_resultat['RESULTAT'].map(urls)
df_resultat = df_resultat[['RESULTAT','URLS']]
df_resultat = df_resultat.reset_index()

In [100]:
df_resultat

,str_result,RESULTAT,URLS
0,20 sachets|vert matcha|the|supreme|bio,"[[pukka-the-vert-matcha-supreme-bio-20-sachets-p100327, gwz, the vert matcha supreme bio 20 sachets], [the-vert-matcha-supreme-20inf, ntr, the vert matcha supreme 20 sachets bio]]","[[https://www.greenweez.com/pukka-the-vert-matcha-supreme-bio-20-sachets-p100327], [https://naturalia.fr/the-vert-matcha-supreme-20inf]]"
1,chocolat lait|100g,"[[saveurs-nature-chocolat-au-lait-100g-p89890, gwz, chocolat lait 100g], [chocolat-lait-100g, ntr, chocolat lait 100g]]","[[https://www.greenweez.com/saveurs-nature-chocolat-au-lait-100g-p89890], [https://naturalia.fr/chocolat-lait-100g]]"
2,couscous demi-complet|500g,"[[markal-couscous-demi-complet-500g-p76290, gwz, couscous demi-complet 500g], [couscous-1-2-cpt-500g, ntr, couscous demi-complet 500g]]","[[https://www.greenweez.com/markal-couscous-demi-complet-500g-p76290], [https://naturalia.fr/couscous-1-2-cpt-500g]]"
3,lentilles vertes|petits legumes|cuisinees,"[[prosain-lentilles-vertes-cuisinees-aux-petits-legumes-p69328, gwz, lentilles vertes cuisinees petits legumes], [8144-lentilles-vertes-bio-cuisinees-aux-petits-legumes-3335880004716?PageSpeed=noscript, wbe, lentilles vertes cuisinees petits legumes], [4083-lentilles-vertes-bio-cuisinees-aux-petits-legumes-3335880006031?PageSpeed=noscript, wbe, lentilles vertes cuisinees petits legumes]]","[[https://www.greenweez.com/prosain-lentilles-vertes-cuisinees-aux-petits-legumes-p69328], [https://www.webecologie.com/conserve-plat-cuisine-bio/8144-lentilles-vertes-bio-cuisinees-aux-petits-legumes-3335880004716.html?PageSpeed=noscript], [https://www.webecologie.com/conserve-plat-cuisine-bio/4083-lentilles-vertes-bio-cuisinees-aux-petits-legumes-3335880006031.html?PageSpeed=noscript]]"
4,sesame noir|250g,"[[markal-sesame-noir-250g-p77439, gwz, sesame noir 250g], [sesame-noir-250g, ntr, sesame noir 250g]]","[[https://www.greenweez.com/markal-sesame-noir-250g-p77439], [https://naturalia.fr/sesame-noir-250g]]"


In [101]:
import pandas as pd
import numpy as np

z = []
for k,row in df_resultat.iterrows():
#     for j in list(np.array(row.b).flat):
#     print (list(np.array(row.RESULTAT_FINAL)))
    print (list(np.array(row.RESULTAT)))
#     for (i,j) in zip(list(np.array(row.RESULTAT_FINAL).flat), list(np.array(row.URLS).flat)):
#     for (i,j) in zip(list(np.array(row.RESULTAT_FINAL)), list(np.array(row.URLS).flat)):
    for (i,j) in zip(list(np.array(row.RESULTAT)), list(np.array(row.URLS).flat)):
#         z.append({'a':row.a, 'b':j})
#         print (row.a,i,j)
#         print (i,j)
#         z.append({'a':row.a, 'b':i, 'c':j})
#         z.append({'str_result':row.str_result,'RESULTAT_FINAL':i, 'URLS':j})
        z.append({'str_result':row.str_result,'RESULTAT':i, 'URLS':j})
#         print (z)
df_resultat = pd.DataFrame(z)
# df_resultat = df_resultat[['str_result','RESULTAT_FINAL','URLS']]
df_resultat = df_resultat[['str_result','RESULTAT','URLS']]
df_resultat = df_resultat.style.format({'URLS': make_clickable})

[array(['pukka-the-vert-matcha-supreme-bio-20-sachets-p100327', 'gwz',
       'the vert matcha supreme bio 20 sachets'], dtype='<U52'), array(['the-vert-matcha-supreme-20inf', 'ntr',
       'the vert matcha supreme 20 sachets bio'], dtype='<U52')]
[array(['saveurs-nature-chocolat-au-lait-100g-p89890', 'gwz',
       'chocolat lait 100g'], dtype='<U43'), array(['chocolat-lait-100g', 'ntr', 'chocolat lait 100g'], dtype='<U43')]
[array(['markal-couscous-demi-complet-500g-p76290', 'gwz',
       'couscous demi-complet 500g'], dtype='<U40'), array(['couscous-1-2-cpt-500g', 'ntr', 'couscous demi-complet 500g'],
      dtype='<U40')]
[array(['prosain-lentilles-vertes-cuisinees-aux-petits-legumes-p69328',
       'gwz', 'lentilles vertes cuisinees petits legumes'], dtype='<U87'), array(['8144-lentilles-vertes-bio-cuisinees-aux-petits-legumes-3335880004716?PageSpeed=noscript',
       'wbe', 'lentilles vertes cuisinees petits legumes'], dtype='<U87'), array(['4083-lentilles-vertes-bio-cuisinees-aux-

In [105]:
df_resultat
# A mettre sur : https://github.com/peteryoda.com

,str_result,RESULTAT,URLS
0,20 sachets|vert matcha|the|supreme|bio,['pukka-the-vert-matcha-supreme-bio-20-sachets-p100327' 'gwz' 'the vert matcha supreme bio 20 sachets'],https://www.greenweez.com/pukka-the-vert-matcha-supreme-bio-20-sachets-p100327
1,20 sachets|vert matcha|the|supreme|bio,['the-vert-matcha-supreme-20inf' 'ntr' 'the vert matcha supreme 20 sachets bio'],https://naturalia.fr/the-vert-matcha-supreme-20inf
2,chocolat lait|100g,['saveurs-nature-chocolat-au-lait-100g-p89890' 'gwz' 'chocolat lait 100g'],https://www.greenweez.com/saveurs-nature-chocolat-au-lait-100g-p89890
3,chocolat lait|100g,['chocolat-lait-100g' 'ntr' 'chocolat lait 100g'],https://naturalia.fr/chocolat-lait-100g
4,couscous demi-complet|500g,['markal-couscous-demi-complet-500g-p76290' 'gwz' 'couscous demi-complet 500g'],https://www.greenweez.com/markal-couscous-demi-complet-500g-p76290
5,couscous demi-complet|500g,['couscous-1-2-cpt-500g' 'ntr' 'couscous demi-complet 500g'],https://naturalia.fr/couscous-1-2-cpt-500g
6,lentilles vertes|petits legumes|cuisinees,['prosain-lentilles-vertes-cuisinees-aux-petits-legumes-p69328' 'gwz' 'lentilles vertes cuisinees petits legumes'],https://www.greenweez.com/prosain-lentilles-vertes-cuisinees-aux-petits-legumes-p69328
7,lentilles vertes|petits legumes|cuisinees,['8144-lentilles-vertes-bio-cuisinees-aux-petits-legumes-3335880004716?PageSpeed=noscript' 'wbe' 'lentilles vertes cuisinees petits legumes'],https://www.webecologie.com/conserve-plat-cuisine-bio/8144-lentilles-vertes-bio-cuisinees-aux-petits-legumes-3335880004716.html?PageSpeed=noscript
8,lentilles vertes|petits legumes|cuisinees,['4083-lentilles-vertes-bio-cuisinees-aux-petits-legumes-3335880006031?PageSpeed=noscript' 'wbe' 'lentilles vertes cuisinees petits legumes'],https://www.webecologie.com/conserve-plat-cuisine-bio/4083-lentilles-vertes-bio-cuisinees-aux-petits-legumes-3335880006031.html?PageSpeed=noscript
9,sesame noir|250g,['markal-sesame-noir-250g-p77439' 'gwz' 'sesame noir 250g'],https://www.greenweez.com/markal-sesame-noir-250g-p77439
